In [2]:
#25-08-2022 Создна копия для SUNLIGHT
#25-08-2022 Для цек БК и ПДК
#07-07-2022 0 обработка нефти 
#06-07-2022 серебро https://miuz.ru/catalog/filter/offers_metall-is-serebro/page=2/
#16-06-2022 Для Алексея Руб-Из-Сап
#24-05-2022 Запуск парсинга result is 'mius240522_0-4154.xlsx'

In [3]:
from bs4 import BeautifulSoup
import requests
from requests import Request, Session
import pandas as pd
import re

def mysoup(urls):
    response = requests.get(urls)
    return BeautifulSoup(response.text, 'lxml')
   

def getlinks_muiz(urls):
    quotes = mysoup(urls).find_all('div', class_="product")
    res = [q.find("a").get("href") for q in quotes]
    print(len(res), 'links is ready ' , res[0], '...', res[-1])
    return res
    

def getlinks_sunl(urls):
    quotes = mysoup(urls).find_all('a', class_="cl-item-link js-cl-item-link js-cl-item-root-link")
    res = [q.get("href") for q in quotes]
    print(len(res), 'links is ready ' , res[0], '...', res[-1])
    return res
    

def get_info_from_card(urls):
    card = mysoup(urls)
    return card.select('body > div.grid > div > div > div.detail__item.detail__item--new > div.detail__item-col._3.position-relative > form > div:nth-child(8)')
    

In [9]:
i=2
f'page={i}/'

'page=2/'

In [10]:
#def getlinks_sunl(urls):
#quotes = mysoup(urls).find_all('a', class_="cl-item-link js-cl-item-link js-cl-item-root-link")
#res = [q.get("href") for q in quotes]
#print(len(res), 'links is ready ' , res[0], '...', res[-1])

In [11]:
#res

## Добываем линки на карточки SUNLIGHT 

In [12]:
links = []

In [14]:
#def main():

#urls = f'https://sunlight.net/catalog/'#все

urls = f"https://sunlight.net/catalog/?stone=1" #без вставки
domain = f'https://sunlight.net'
links += getlinks_sunl(urls)
for i in range(2,148):
    #pages = f'page-{i}/'#все 
    pages = f'&page={i}'#без вставки

    print(i, urls+pages)
    links += getlinks_sunl(urls+pages)
    

60 links is ready  /catalog/earring_271510.html ... /catalog/ring_60754.html
2 https://sunlight.net/catalog/?stone=1&page=2
60 links is ready  /catalog/pendants_74404.html ... /catalog/earring_306791.html
3 https://sunlight.net/catalog/?stone=1&page=3
60 links is ready  /catalog/ring_263294.html ... /catalog/neck_decoration_328983.html
4 https://sunlight.net/catalog/?stone=1&page=4
60 links is ready  /catalog/earring_83390.html ... /catalog/ring_321887.html
5 https://sunlight.net/catalog/?stone=1&page=5
60 links is ready  /catalog/bracelets_87777.html ... /catalog/earring_84729.html
6 https://sunlight.net/catalog/?stone=1&page=6
60 links is ready  /catalog/cutlery_254024.html ... /catalog/earring_285100.html
7 https://sunlight.net/catalog/?stone=1&page=7
60 links is ready  /catalog/earring_278946.html ... /catalog/ring_263125.html
8 https://sunlight.net/catalog/?stone=1&page=8
60 links is ready  /catalog/bracelets_330687.html ... /catalog/bracelets_71488.html
9 https://sunlight.net/cat

60 links is ready  /catalog/ring_259125.html ... /catalog/clock_258848.html
66 https://sunlight.net/catalog/?stone=1&page=66
60 links is ready  /catalog/chains_324492.html ... /catalog/chains_189576.html
67 https://sunlight.net/catalog/?stone=1&page=67
60 links is ready  /catalog/neck_decoration_108754.html ... /catalog/tie-clips_256276.html
68 https://sunlight.net/catalog/?stone=1&page=68
60 links is ready  /catalog/earring_92199.html ... /catalog/bracelets_143812.html
69 https://sunlight.net/catalog/?stone=1&page=69
60 links is ready  /catalog/bracelets_127384.html ... /catalog/bracelets_91991.html
70 https://sunlight.net/catalog/?stone=1&page=70
60 links is ready  /catalog/bracelets_87521.html ... /catalog/earring_267527.html
71 https://sunlight.net/catalog/?stone=1&page=71
60 links is ready  /catalog/earring_268803.html ... /catalog/pendants_284144.html
72 https://sunlight.net/catalog/?stone=1&page=72
60 links is ready  /catalog/earring_234188.html ... /catalog/clock_279466.html
73

60 links is ready  /catalog/pendants_260949.html ... /catalog/earring_263555.html
129 https://sunlight.net/catalog/?stone=1&page=129
60 links is ready  /catalog/pendants_261022.html ... /catalog/one_earring_273281.html
130 https://sunlight.net/catalog/?stone=1&page=130
60 links is ready  /catalog/earring_273269.html ... /catalog/earring_245013.html
131 https://sunlight.net/catalog/?stone=1&page=131
60 links is ready  /catalog/earring_244919.html ... /catalog/ring_278699.html
132 https://sunlight.net/catalog/?stone=1&page=132
60 links is ready  /catalog/ring_278674.html ... /catalog/ring_130366.html
133 https://sunlight.net/catalog/?stone=1&page=133
60 links is ready  /catalog/ring_130355.html ... /catalog/earring_92003.html
134 https://sunlight.net/catalog/?stone=1&page=134
60 links is ready  /catalog/ring_90568.html ... /catalog/earring_258297.html
135 https://sunlight.net/catalog/?stone=1&page=135
60 links is ready  /catalog/earring_258271.html ... /catalog/ring_129954.html
136 https

In [344]:
a = "https://sunlight.net/catalog/?stone=1&page=2"
b = "https://sunlight.net/catalog/?stone=1&pagе=2" #my
len(b)
[a[i]==b[i] for i in range(len(a))]
id('e'), id('e')

(1651384493936, 1651384493936)

In [343]:
for i in range(len(a)):
    print(a[i],b[i], a[i]==b[i])


h h True
t t True
t t True
p p True
s s True
: : True
/ / True
/ / True
s s True
u u True
n n True
l l True
i i True
g g True
h h True
t t True
. . True
n n True
e e True
t t True
/ / True
c c True
a a True
t t True
a a True
l l True
o o True
g g True
/ / True
? ? True
s s True
t t True
o o True
n n True
e e True
= = True
1 1 True
& & True
p p True
a a True
g g True
e е False
= = True
2 2 True


In [17]:
len(links)

8810

In [16]:
#drop duplicates subset keep
links = list(set(links))

In [116]:
links

['/catalog/ring_282786.html',
 '/catalog/neck_decoration_209254.html',
 '/catalog/pendants_252289.html',
 '/catalog/bracelets_45278.html',
 '/catalog/earring_246168.html',
 '/catalog/bracelets_165862.html',
 '/catalog/bracelets_257854.html',
 '/catalog/neck_decoration_79978.html',
 '/catalog/earring_265968.html',
 '/catalog/earring_292054.html',
 '/catalog/pendants_75760.html',
 '/catalog/earring_259829.html',
 '/catalog/ring_251029.html',
 '/catalog/ring_332303.html',
 '/catalog/bracelets_79506.html',
 '/catalog/clock_87636.html',
 '/catalog/bracelets_257562.html',
 '/catalog/necklace_100485.html',
 '/catalog/pendants_89562.html',
 '/catalog/neck_decoration_264731.html',
 '/catalog/clock_276727.html',
 '/catalog/earring_307692.html',
 '/catalog/neck_decoration_246380.html',
 '/catalog/earring_268902.html',
 '/catalog/bracelets_158427.html',
 '/catalog/earring_89029.html',
 '/catalog/earring_276969.html',
 '/catalog/earring_299557.html',
 '/catalog/ring_90737.html',
 '/catalog/ring_232

In [117]:
domain+links[0]

'https://sunlight.net/catalog/ring_282786.html'

## Словарь линков (save-load)

In [61]:
taskname, taskdescr =  'бескаменка300822', 'все БК  8810'

In [63]:
import json

In [64]:
import datetime
datestr = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
datestr

'2022-08-30-11-18'

In [73]:
taskname, taskdescr

('бескаменка300822', 'все БК  8810')

In [65]:
#first time
dlinks = {'name':['datestr','links','urls','descr'],
          taskname:[datestr, links,urls,taskdescr]}

In [68]:
with open(folderlink+'dlinks_sunlight'+datestr+'.json', 'w') as writefile:
    json.dump(dlinks, writefile)

In [99]:
folderlink = 'links/'
files = os.listdir(folderlink)
print([e for e in enumerate(files)])

[(0, 'dlinks_sunlight2022-08-30-11-18.json')]


In [75]:
with open(folderlink+files[0], 'r') as writefile:
    dlinks = json.load(writefile)

In [78]:
dlinks['name']

['datestr', 'links', 'urls', 'descr']

In [82]:
links = dlinks[taskname][1]
len(links)

8810

## Разработка парсера по карточкам

In [70]:
#urls = 'https://sunlight.net/catalog/page-3/'

'https://sunlight.net/catalog/ring_165495.html'

In [72]:
mysoup(urls)

In [35]:
#def getlinks(urls):
quotes = mysoup(urls).find_all('a', class_="cl-item-link js-cl-item-link js-cl-item-root-link")

In [36]:
len(quotes)

60

In [37]:
quotes[1].get("href")

'/catalog/earring_54875.html'

In [38]:
res = [q.get("href") for q in quotes]
#    print(len(res), 'links is ready ' , res[0], '...', res[-1])
#    return res
    

In [39]:
res

['/catalog/pendants_63494.html',
 '/catalog/earring_54875.html',
 '/catalog/ring_313307.html',
 '/catalog/earring_92268.html',
 '/catalog/pendants_267177.html',
 '/catalog/earring_241202.html',
 '/catalog/silver/chains206837.html',
 '/catalog/ring_187860.html',
 '/catalog/ring_75713.html',
 '/catalog/pendants_90939.html',
 '/catalog/earring_266753.html',
 '/catalog/ring_47781.html',
 '/catalog/ring_83896.html',
 '/catalog/earring_69241.html',
 '/catalog/ring_46699.html',
 '/catalog/neck_decoration_301834.html',
 '/catalog/ring_100205.html',
 '/catalog/anklets_329109.html',
 '/catalog/ring_78036.html',
 '/catalog/earring_81222.html',
 '/catalog/silver/chains223726.html',
 '/catalog/ring_49414.html',
 '/catalog/earring_241347.html',
 '/catalog/silver/earring226184.html',
 '/catalog/silver/pendants224757.html',
 '/catalog/lace_327443.html',
 '/catalog/pendants_70833.html',
 '/catalog/ring_66705.html',
 '/catalog/ring_187859.html',
 '/catalog/one_earring_327325.html',
 '/catalog/earring_18

In [17]:
<a href="/catalog/pendants_90939.html" class="cl-item-link js-cl-item-link js-cl-item-root-link" itemprop="url">
                    <span itemprop="name">Подвеска, вставка: фианит; наносапфир; Серебро 925 пробы.</span>
                </a>


In [ ]:
<a href="/catalog/ring_47781.html" class="cl-item-link js-cl-item-link js-cl-item-root-link" itemprop="url">
                    <span itemprop="name">Кольцо, вставка: фианит; Серебро 925 пробы.</span>
                </a>

## Собранная функция парсера по карточкам

### Версия 29-08-2022 SUNLIGHT

In [101]:

def parcing(links): 

    for i in range(3000):#links)):
        try:
            row = len(data)
            urlcard = domain +links[i]
            card = mysoup(urlcard)
            data.loc[row,'url'] = urlcard
            data.loc[row,'h1'] = card.find("h1").text.strip()
            data.loc[row,'art'] = card.find('div', class_="supreme-product-card__product-article-text").text.strip()
            data.loc[row,'price'] = card.find('div', class_="supreme-product-card__price-discount-price").text.strip().replace('\u202f', '').replace('\xa0', '')
            data.loc[row,'price2'] = card.find('div', class_="supreme-product-card__price-old").text.strip().replace('\u202f', '').replace('\xa0', '')
            a = card.find_all('p', class_="supreme-product-card__description supreme-product-card__description_default")
            data.loc[row,'gold'] = ';'.join([x.text for x in a])
            data.loc[row,'d1'] = "&".join([t.text for t in card.find_all('span', class_="supreme-product-card-description__item-text")])
            data.loc[row,'d0'] = "&".join([t.text for t in card.find_all('a', class_="supreme-product-card-description__item-text")])
            #print(i, urlcard)
        except (AttributeError,TypeError) as e:
            print('error',e)
        except IndexError:
            print('indexerror len of = ' , len(card.find_all('div','detail__item-option')) )
        print(i, urlcard)
    
def byrepost():

    def gold(t,patt):
        f = re.search(patt,t)
        return f.group() if f else 're не найдено'
    
    pattern_gold = r'[Сс]ереб\w+|[Зз]олот\w*|[Сс]таль'
    pattern_diam = r'[Бб]риллиант\w*'
    pattern_probe = r'[\d]{,3}(?=\s*проб)'
    pattern_gramm = r'[\d.\d]{,5}(?=г;)'
    pattern_vst = r'вставк\w*'#r'вставк\w*'
    pattern_type = r'(?<=;)\w+\s?\w*'        
        
    
    data['diam'] = data.gold.apply(lambda x: gold(x, pattern_diam))
    data['goldsilv'] = data.h1.apply(lambda x: gold(x, pattern_gold))
    data['probe'] = data.gold.apply(lambda x: gold(x, pattern_probe))
    data['gramm'] = data.gold.apply(lambda x: gold(x, pattern_gramm))
    data['vst'] = data.gold.apply(lambda x: gold(x, pattern_vst))
    data['type'] = data.gold.apply(lambda x: gold(x, pattern_type))

In [100]:
data = pd.DataFrame(columns=['h1','art','price','price2','gold','gold2','weight','gems','gems2','url'])


In [102]:
parcing(links)

0 https://sunlight.net/catalog/ring_282786.html
1 https://sunlight.net/catalog/neck_decoration_209254.html
2 https://sunlight.net/catalog/pendants_252289.html
3 https://sunlight.net/catalog/bracelets_45278.html
4 https://sunlight.net/catalog/earring_246168.html
5 https://sunlight.net/catalog/bracelets_165862.html
6 https://sunlight.net/catalog/bracelets_257854.html
7 https://sunlight.net/catalog/neck_decoration_79978.html
8 https://sunlight.net/catalog/earring_265968.html
9 https://sunlight.net/catalog/earring_292054.html
10 https://sunlight.net/catalog/pendants_75760.html
11 https://sunlight.net/catalog/earring_259829.html
12 https://sunlight.net/catalog/ring_251029.html
13 https://sunlight.net/catalog/ring_332303.html
14 https://sunlight.net/catalog/bracelets_79506.html
15 https://sunlight.net/catalog/clock_87636.html
16 https://sunlight.net/catalog/bracelets_257562.html
17 https://sunlight.net/catalog/necklace_100485.html
18 https://sunlight.net/catalog/pendants_89562.html
19 https:

155 https://sunlight.net/catalog/chains_324493.html
156 https://sunlight.net/catalog/clock_285981.html
157 https://sunlight.net/catalog/chains_81626.html
158 https://sunlight.net/catalog/earring_246463.html
159 https://sunlight.net/catalog/neck_decoration_163371.html
160 https://sunlight.net/catalog/bracelets_79015.html
161 https://sunlight.net/catalog/earring_263555.html
162 https://sunlight.net/catalog/earring_260989.html
163 https://sunlight.net/catalog/earring_247034.html
164 https://sunlight.net/catalog/bracelets_91888.html
165 https://sunlight.net/catalog/ring_233347.html
166 https://sunlight.net/catalog/earring_99787.html
167 https://sunlight.net/catalog/ring_236167.html
168 https://sunlight.net/catalog/bracelets_71488.html
169 https://sunlight.net/catalog/earring_273360.html
170 https://sunlight.net/catalog/pendants_158928.html
171 https://sunlight.net/catalog/pendants_165581.html
172 https://sunlight.net/catalog/clock_315529.html
173 https://sunlight.net/catalog/bracelets_3306

309 https://sunlight.net/catalog/bracelets_127839.html
310 https://sunlight.net/catalog/clock_290282.html
311 https://sunlight.net/catalog/earring_273153.html
312 https://sunlight.net/catalog/clock_290289.html
313 https://sunlight.net/catalog/earring_258263.html
314 https://sunlight.net/catalog/clock_315435.html
315 https://sunlight.net/catalog/ring_301526.html
316 https://sunlight.net/catalog/earring_329455.html
317 https://sunlight.net/catalog/one_earring_267167.html
318 https://sunlight.net/catalog/earring_244017.html
319 https://sunlight.net/catalog/neck_decoration_311556.html
320 https://sunlight.net/catalog/earring_109602.html
321 https://sunlight.net/catalog/clock_258852.html
322 https://sunlight.net/catalog/neck_decoration_263856.html
323 https://sunlight.net/catalog/neck_decoration_108754.html
324 https://sunlight.net/catalog/clock_313416.html
325 https://sunlight.net/catalog/ring_241728.html
326 https://sunlight.net/catalog/pendants_103704.html
327 https://sunlight.net/catalo

463 https://sunlight.net/catalog/earring_260982.html
464 https://sunlight.net/catalog/earring_92337.html
465 https://sunlight.net/catalog/ring_265842.html
466 https://sunlight.net/catalog/bracelets_301602.html
467 https://sunlight.net/catalog/earring_307471.html
468 https://sunlight.net/catalog/ring_263298.html
469 https://sunlight.net/catalog/earring_314523.html
470 https://sunlight.net/catalog/earring_217498.html
471 https://sunlight.net/catalog/earring_248918.html
472 https://sunlight.net/catalog/pendants_242682.html
473 https://sunlight.net/catalog/pendants_271483.html
474 https://sunlight.net/catalog/ring_298926.html
475 https://sunlight.net/catalog/clock_315434.html
476 https://sunlight.net/catalog/earring_256883.html
477 https://sunlight.net/catalog/earring_82643.html
478 https://sunlight.net/catalog/earring_99259.html
479 https://sunlight.net/catalog/ring_247258.html
480 https://sunlight.net/catalog/bracelets_108194.html
481 https://sunlight.net/catalog/earring_158432.html
482 

618 https://sunlight.net/catalog/earring_249592.html
619 https://sunlight.net/catalog/clock_290496.html
620 https://sunlight.net/catalog/earring_246438.html
621 https://sunlight.net/catalog/pendants_80722.html
622 https://sunlight.net/catalog/pendants_93913.html
623 https://sunlight.net/catalog/earring_249098.html
624 https://sunlight.net/catalog/earring_314500.html
625 https://sunlight.net/catalog/pendants_271400.html
626 https://sunlight.net/catalog/earring_318776.html
627 https://sunlight.net/catalog/pendants_330846.html
628 https://sunlight.net/catalog/earring_309499.html
629 https://sunlight.net/catalog/pendants_267935.html
630 https://sunlight.net/catalog/earring_128452.html
631 https://sunlight.net/catalog/ring_188242.html
632 https://sunlight.net/catalog/earring_109505.html
633 https://sunlight.net/catalog/earring_266530.html
634 https://sunlight.net/catalog/bracelets_330908.html
635 https://sunlight.net/catalog/earring_111406.html
636 https://sunlight.net/catalog/earring_92632

...

2296 https://sunlight.net/catalog/ring_260969.html
2297 https://sunlight.net/catalog/neck_decoration_263004.html
2298 https://sunlight.net/catalog/earring_246340.html
2299 https://sunlight.net/catalog/earring_242489.html
2300 https://sunlight.net/catalog/earring_247706.html
2301 https://sunlight.net/catalog/bracelets_272901.html
2302 https://sunlight.net/catalog/earring_301763.html
2303 https://sunlight.net/catalog/earring_273315.html
2304 https://sunlight.net/catalog/ring_272063.html
2305 https://sunlight.net/catalog/pendants_320704.html
2306 https://sunlight.net/catalog/earring_233333.html
2307 https://sunlight.net/catalog/neck_decoration_262110.html
2308 https://sunlight.net/catalog/earring_236092.html
2309 https://sunlight.net/catalog/bracelets_271728.html
2310 https://sunlight.net/catalog/earring_96999.html
2311 https://sunlight.net/catalog/neck_decoration_253377.html
2312 https://sunlight.net/catalog/earring_249186.html
2313 https://sunlight.net/catalog/bracelets_268713.html
2314

2446 https://sunlight.net/catalog/ring_108511.html
2447 https://sunlight.net/catalog/ring_209262.html
2448 https://sunlight.net/catalog/bracelets_87522.html
2449 https://sunlight.net/catalog/earring_314496.html
2450 https://sunlight.net/catalog/bracelets_91787.html
2451 https://sunlight.net/catalog/bracelets_253803.html
2452 https://sunlight.net/catalog/earring_154301.html
2453 https://sunlight.net/catalog/neck_decoration_239413.html
2454 https://sunlight.net/catalog/earring_66237.html
2455 https://sunlight.net/catalog/clock_331948.html
2456 https://sunlight.net/catalog/bracelets_151996.html
2457 https://sunlight.net/catalog/neck_decoration_247594.html
2458 https://sunlight.net/catalog/one_earring_271970.html
2459 https://sunlight.net/catalog/ring_120229.html
2460 https://sunlight.net/catalog/neck_decoration_110213.html
2461 https://sunlight.net/catalog/earring_82539.html
2462 https://sunlight.net/catalog/pendants_99984.html
2463 https://sunlight.net/catalog/earring_272463.html
2464 ht

2597 https://sunlight.net/catalog/bracelets_287384.html
2598 https://sunlight.net/catalog/ring_126678.html
2599 https://sunlight.net/catalog/pendants_234867.html
2600 https://sunlight.net/catalog/earring_105111.html
2601 https://sunlight.net/catalog/bracelets_265889.html
2602 https://sunlight.net/catalog/pendants_277653.html
2603 https://sunlight.net/catalog/clock_83422.html
2604 https://sunlight.net/catalog/bracelets_259821.html
2605 https://sunlight.net/catalog/earring_261178.html
2606 https://sunlight.net/catalog/pendants_264548.html
2607 https://sunlight.net/catalog/cufflinks_92076.html
2608 https://sunlight.net/catalog/earring_246314.html
2609 https://sunlight.net/catalog/bracelets_251583.html
2610 https://sunlight.net/catalog/bracelets_331314.html
2611 https://sunlight.net/catalog/earring_124850.html
2612 https://sunlight.net/catalog/earring_103992.html
2613 https://sunlight.net/catalog/earring_268571.html
2614 https://sunlight.net/catalog/bracelets_245573.html
2615 https://sunli

2748 https://sunlight.net/catalog/ring_328037.html
2749 https://sunlight.net/catalog/silver/earring116865.html
2750 https://sunlight.net/catalog/earring_272025.html
2751 https://sunlight.net/catalog/bracelets_246243.html
2752 https://sunlight.net/catalog/pendants_116539.html
2753 https://sunlight.net/catalog/earring_271409.html
2754 https://sunlight.net/catalog/earring_109461.html
2755 https://sunlight.net/catalog/ring257850.html
2756 https://sunlight.net/catalog/earring_206290.html
2757 https://sunlight.net/catalog/earring_273374.html
2758 https://sunlight.net/catalog/pendants_277171.html
2759 https://sunlight.net/catalog/earring_273825.html
2760 https://sunlight.net/catalog/neck_decoration_251427.html
2761 https://sunlight.net/catalog/pendants_272924.html
2762 https://sunlight.net/catalog/earring_242153.html
2763 https://sunlight.net/catalog/bracelets_238712.html
2764 https://sunlight.net/catalog/pendants_58725.html
2765 https://sunlight.net/catalog/tie-clips_157662.html
2766 https:/

2898 https://sunlight.net/catalog/earring_249496.html
2899 https://sunlight.net/catalog/bracelets_218419.html
2900 https://sunlight.net/catalog/bracelets_251591.html
2901 https://sunlight.net/catalog/pendants_290608.html
2902 https://sunlight.net/catalog/ring_273407.html
2903 https://sunlight.net/catalog/earring_83480.html
2904 https://sunlight.net/catalog/silver/necklaces257886.html
2905 https://sunlight.net/catalog/bracelets_282001.html
2906 https://sunlight.net/catalog/ring_254860.html
2907 https://sunlight.net/catalog/earring_312630.html
2908 https://sunlight.net/catalog/earring_267024.html
2909 https://sunlight.net/catalog/earring_97263.html
2910 https://sunlight.net/catalog/earring_304127.html
2911 https://sunlight.net/catalog/decor_106217.html
2912 https://sunlight.net/catalog/earring_243990.html
2913 https://sunlight.net/catalog/clock_290450.html
2914 https://sunlight.net/catalog/earring_256046.html
2915 https://sunlight.net/catalog/earring_218418.html
2916 https://sunlight.net

In [105]:
byrepost()

In [106]:
data

,h1,art,price,price2,gold,gold2,weight,gems,gems2,url,d1,d0,diam,goldsilv,probe,gramm,vst
0,Золотое кольцо,Артикул: 282786,10080₽,33600₽,Артикул: 282786;Кольцо \nРозовое золото 585...,NaN,NaN,NaN,NaN,https://sunlight.net/catalog/ring_282786.html,10355а-2*&РОССИЯ&Розовое золото&1.6 г,SERGEY GRIBNYAKOV&Золотые украшения без вставок,re не найдено,Золотое,585,1.6,re не найдено
1,Стальное колье,Артикул: 209254,414₽,1380₽,"Артикул: 209254;Шейное укр., вставка: Сталь; ...",NaN,NaN,NaN,NaN,https://sunlight.net/catalog/neck_decoration_2...,Q9068-H0B-01&Ювелирная сталь,OKAMI,re не найдено,Сталь,re не найдено,re не найдено,вставка
2,"Серебряная подвеска ""Сергий Радонежский""",Артикул: 252289,414₽,1380₽,Артикул: 252289;Подвеска \nСеребро 925 проб...,NaN,NaN,NaN,NaN,https://sunlight.net/catalog/pendants_252289.html,04-011&РОССИЯ&Серебро&1.45 г,Епархия&Христианство,re не найдено,Серебряная,925,1.45,re не найдено
3,Золотой браслет,Артикул: 45278,9140₽,30467₽,Артикул: 45278;Браслет \nРозовое золото 585...,NaN,NaN,NaN,NaN,https://sunlight.net/catalog/bracelets_45278.html,"7050154-18*&Розовое золото&3,2 мм&2.54 г",LINEA&Цепи Полновесные,re не найдено,Золотой,585,2.54,re не найдено
4,Серебряные серьги,Артикул: 246168,2790₽,5580₽,Артикул: 246168;Серьги \nСеребро 925 пробы....,NaN,NaN,NaN,NaN,https://sunlight.net/catalog/earring_246168.html,S13378-C9W-01&Серебро&1.65 г,SERGEY GRIBNYAKOV&СЕРЕБРО. FASHION,re не найдено,Серебряные,925,1.65,re не найдено
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,"Серебряная подвеска ""Ангел Хранитель""",Артикул: 88720,534₽,1780₽,Артикул: 88720;Подвеска \nСеребро 925 пробы...,NaN,NaN,NaN,NaN,https://sunlight.net/catalog/pendants_88720.html,5401-084&РОССИЯ&Серебро&1.98 г,SERGEY GRIBNYAKOV&Христианство,re не найдено,Серебряная,925,1.98,re не найдено
2996,Золотые серьги,Артикул: 274034,4194₽,13980₽,Артикул: 274034;Серьги \nРозовое золото 585...,NaN,NaN,NaN,NaN,https://sunlight.net/catalog/earring_274034.html,12526-C5R-01&Розовое золото&0.63 г,SERGEY GRIBNYAKOV&Пирсинг Золото,re не найдено,Золотые,585,0.63,re не найдено
2997,Часы женские,Артикул: 315484,2514₽,8380₽,"Артикул: 315484;Функции: Время, секунда, Дата\...",NaN,NaN,NaN,NaN,https://sunlight.net/catalog/clock_315484.html,KM35SSN-15LN&Натуральная кожа&Япония&35 мм&Мин...,OKAMI&Нержавеющая сталь 316L&Женщинам,re не найдено,re не найдено,re не найдено,re не найдено,re не найдено
2998,Серебряный браслет,Артикул: 257717,2490₽,8300₽,Артикул: 257717;Браслет \nСеребро 925 пробы...,NaN,NaN,NaN,NaN,https://sunlight.net/catalog/bracelets_257717....,БМ100Г&РОССИЯ&Серебро&16.77 г,SUNLIGHT,re не найдено,Серебряный,925,16.77,re не найдено


In [428]:
#data = pd.DataFrame(columns=['h1','art','price','price2','gold','gold2','weight','gems','gems2','url'])

In [429]:
#29-08-2022 for sunlight 6000
#25-08-2022 all
#06-07-2022 серебро
#16-06-2022 Для Алексея Сапфир Изумруд Рубин
#24-05-2022
#14-04-2022

for i in range(len(links)):
    try:
        row = len(data)
        urlcard = domain +links[i]
        card = mysoup(urlcard)
        data.loc[row,'url'] = urlcard
        data.loc[row,'h1'] = card.find("h1").text.strip()
        data.loc[row,'art'] = card.find('div', class_="supreme-product-card__product-article-text").text.strip()
        data.loc[row,'price'] = card.find('div', class_="supreme-product-card__price-discount-price").text.strip().replace('\u202f', '').replace('\xa0', '')
        data.loc[row,'price2'] = card.find('div', class_="supreme-product-card__price-old").text.strip().replace('\u202f', '').replace('\xa0', '')
        a = card.find_all('p', class_="supreme-product-card__description supreme-product-card__description_default")
        data.loc[row,'gold'] = ';'.join([x.text for x in a])
        data.loc[row,'d1'] = "&".join([t.text for t in card.find_all('span', class_="supreme-product-card-description__item-text")])
        data.loc[row,'d0'] = "&".join([t.text for t in card.find_all('a', class_="supreme-product-card-description__item-text")])
        #print(i, urlcard)
    except (AttributeError,TypeError) as e:
        print('error',e)
    except IndexError:
        print('indexerror len of = ' , len(card.find_all('div','detail__item-option')) )
    print(i, urlcard)


0 https://sunlight.net/catalog/clock_263766.html
1 https://sunlight.net/catalog/ring_114165.html
2 https://sunlight.net/catalog/earring_273106.html
3 https://sunlight.net/catalog/earring_88536.html
4 https://sunlight.net/catalog/pendants_40983.html
5 https://sunlight.net/catalog/earring_87492.html
6 https://sunlight.net/catalog/earring_245239.html
7 https://sunlight.net/catalog/earring_89063.html
8 https://sunlight.net/catalog/clock_290342.html
9 https://sunlight.net/catalog/ring_89563.html
10 https://sunlight.net/catalog/earring_268869.html
11 https://sunlight.net/catalog/neck_decoration_286148.html
12 https://sunlight.net/catalog/earring_267786.html
13 https://sunlight.net/catalog/piercing_276190.html
14 https://sunlight.net/catalog/pendants_315142.html
15 https://sunlight.net/catalog/earring_274553.html
16 https://sunlight.net/catalog/earring_273317.html
17 https://sunlight.net/catalog/earring_248698.html
18 https://sunlight.net/catalog/bracelets_306324.html
19 https://sunlight.net/

157 https://sunlight.net/catalog/earring_153790.html
158 https://sunlight.net/catalog/earring_285374.html
159 https://sunlight.net/catalog/neck_decoration_313821.html
160 https://sunlight.net/catalog/earring_263345.html
161 https://sunlight.net/catalog/neck_decoration_309495.html
162 https://sunlight.net/catalog/earring_268509.html
163 https://sunlight.net/catalog/earring_104065.html
164 https://sunlight.net/catalog/neck_decoration_88952.html
165 https://sunlight.net/catalog/earring_249497.html
166 https://sunlight.net/catalog/neck_decoration_266014.html
167 https://sunlight.net/catalog/earring_104660.html
168 https://sunlight.net/catalog/pendants_300305.html
169 https://sunlight.net/catalog/pendants_305908.html
170 https://sunlight.net/catalog/pendants_120334.html
171 https://sunlight.net/catalog/earring_261373.html
172 https://sunlight.net/catalog/earring_307471.html
173 https://sunlight.net/catalog/earring_257757.html
174 https://sunlight.net/catalog/earring_281096.html
175 https://

310 https://sunlight.net/catalog/earring_287362.html
311 https://sunlight.net/catalog/bracelets_163392.html
312 https://sunlight.net/catalog/pendants_292270.html
313 https://sunlight.net/catalog/ring_99687.html
314 https://sunlight.net/catalog/earring_99788.html
315 https://sunlight.net/catalog/neck_decoration_273323.html
316 https://sunlight.net/catalog/earring_188877.html
317 https://sunlight.net/catalog/earring_86199.html
318 https://sunlight.net/catalog/neck_decoration_233102.html
319 https://sunlight.net/catalog/earring_238497.html
320 https://sunlight.net/catalog/earring_248910.html
321 https://sunlight.net/catalog/neck_decoration_259118.html
322 https://sunlight.net/catalog/earring_94476.html
323 https://sunlight.net/catalog/ring_264389.html
324 https://sunlight.net/catalog/ring_264209.html
325 https://sunlight.net/catalog/bracelets_238739.html
326 https://sunlight.net/catalog/clock_290315.html
327 https://sunlight.net/catalog/bracelets_277212.html
328 https://sunlight.net/catal

464 https://sunlight.net/catalog/earring_242151.html
465 https://sunlight.net/catalog/ring_246931.html
466 https://sunlight.net/catalog/bracelets_330687.html
467 https://sunlight.net/catalog/pendants_271284.html
468 https://sunlight.net/catalog/earring_242489.html
469 https://sunlight.net/catalog/bracelets_271770.html
470 https://sunlight.net/catalog/bracelets_100159.html
471 https://sunlight.net/catalog/souvenir_99412.html
472 https://sunlight.net/catalog/earring_267528.html
473 https://sunlight.net/catalog/clock_312130.html
474 https://sunlight.net/catalog/bracelets_89147.html
475 https://sunlight.net/catalog/earring_234499.html
476 https://sunlight.net/catalog/earring_307692.html
477 https://sunlight.net/catalog/pendants_327997.html
478 https://sunlight.net/catalog/ring_250503.html
479 https://sunlight.net/catalog/bracelets_238731.html
480 https://sunlight.net/catalog/earring_272023.html
481 https://sunlight.net/catalog/earring_278059.html
482 https://sunlight.net/catalog/neck_decor

618 https://sunlight.net/catalog/pendants_86671.html
619 https://sunlight.net/catalog/bracelets_324483.html
620 https://sunlight.net/catalog/ring_112593.html
621 https://sunlight.net/catalog/earring_143929.html
622 https://sunlight.net/catalog/earring_278154.html
623 https://sunlight.net/catalog/pendants_305893.html
624 https://sunlight.net/catalog/bracelets_52887.html
625 https://sunlight.net/catalog/clock_320492.html
626 https://sunlight.net/catalog/ring_73592.html
627 https://sunlight.net/catalog/neck_decoration_112479.html
628 https://sunlight.net/catalog/bracelets_238827.html
629 https://sunlight.net/catalog/neck_decoration_259510.html
630 https://sunlight.net/catalog/ring_119168.html
631 https://sunlight.net/catalog/ring_308811.html
632 https://sunlight.net/catalog/chains_92621.html
633 https://sunlight.net/catalog/earring_253750.html
634 https://sunlight.net/catalog/earring_299556.html
635 https://sunlight.net/catalog/one_earring_331175.html
636 https://sunlight.net/catalog/brac

...

8183 https://sunlight.net/catalog/bracelets_271764.html
8184 https://sunlight.net/catalog/pendants_272856.html
8185 https://sunlight.net/catalog/cufflinks_259113.html
8186 https://sunlight.net/catalog/neck_decoration_309900.html
8187 https://sunlight.net/catalog/earring_157515.html
8188 https://sunlight.net/catalog/earring_100471.html
8189 https://sunlight.net/catalog/earring_147067.html
8190 https://sunlight.net/catalog/earring_276078.html
8191 https://sunlight.net/catalog/brooch_277890.html
8192 https://sunlight.net/catalog/earring_285327.html
8193 https://sunlight.net/catalog/clock_88059.html
8194 https://sunlight.net/catalog/ring_109432.html
8195 https://sunlight.net/catalog/ring_232765.html
8196 https://sunlight.net/catalog/earring_268998.html
8197 https://sunlight.net/catalog/earring_97785.html
8198 https://sunlight.net/catalog/ring_276981.html
8199 https://sunlight.net/catalog/ring_92762.html
8200 https://sunlight.net/catalog/earring_242158.html
8201 https://sunlight.net/catalog

8334 https://sunlight.net/catalog/earring_245322.html
8335 https://sunlight.net/catalog/earring_234202.html
8336 https://sunlight.net/catalog/earring_308208.html
8337 https://sunlight.net/catalog/pendants_236194.html
8338 https://sunlight.net/catalog/earring_92337.html
8339 https://sunlight.net/catalog/earring_257770.html
8340 https://sunlight.net/catalog/earring_46719.html
8341 https://sunlight.net/catalog/silver/ring254484.html
8342 https://sunlight.net/catalog/earring_263727.html
8343 https://sunlight.net/catalog/pendants_261023.html
8344 https://sunlight.net/catalog/earring_267527.html
8345 https://sunlight.net/catalog/bracelets_264666.html
8346 https://sunlight.net/catalog/earring_242309.html
8347 https://sunlight.net/catalog/earring_87589.html
8348 https://sunlight.net/catalog/bracelets_264669.html
8349 https://sunlight.net/catalog/ring_279237.html
8350 https://sunlight.net/catalog/earring_155071.html
8351 https://sunlight.net/catalog/earring_267752.html
8352 https://sunlight.net

8486 https://sunlight.net/catalog/earring_88705.html
8487 https://sunlight.net/catalog/earring_79697.html
8488 https://sunlight.net/catalog/ring_251011.html
8489 https://sunlight.net/catalog/earring_90780.html
8490 https://sunlight.net/catalog/clock_262558.html
8491 https://sunlight.net/catalog/earring_256051.html
8492 https://sunlight.net/catalog/neck_decoration_259509.html
8493 https://sunlight.net/catalog/bracelets_238818.html
8494 https://sunlight.net/catalog/bracelets_238763.html
8495 https://sunlight.net/catalog/clock_301703.html
8496 https://sunlight.net/catalog/ring_88916.html
8497 https://sunlight.net/catalog/clock_258552.html
8498 https://sunlight.net/catalog/neck_decoration_108195.html
8499 https://sunlight.net/catalog/brooch_128535.html
8500 https://sunlight.net/catalog/bracelets_87928.html
8501 https://sunlight.net/catalog/ring_300201.html
8502 https://sunlight.net/catalog/bracelets_86727.html
8503 https://sunlight.net/catalog/pendants_325844.html
8504 https://sunlight.net

8637 https://sunlight.net/catalog/earring_247630.html
8638 https://sunlight.net/catalog/one_earring_271976.html
8639 https://sunlight.net/catalog/earring_111405.html
8640 https://sunlight.net/catalog/clock_263812.html
8641 https://sunlight.net/catalog/pendants_46825.html
8642 https://sunlight.net/catalog/clock_318266.html
8643 https://sunlight.net/catalog/earring_258223.html
8644 https://sunlight.net/catalog/bracelets_261204.html
8645 https://sunlight.net/catalog/earring_287380.html
8646 https://sunlight.net/catalog/pendants_118718.html
8647 https://sunlight.net/catalog/neck_decoration_259804.html
8648 https://sunlight.net/catalog/bracelets_274189.html
8649 https://sunlight.net/catalog/earring_246459.html
8650 https://sunlight.net/catalog/pendants_273255.html
8651 https://sunlight.net/catalog/necklace_119470.html
8652 https://sunlight.net/catalog/one_earring_153918.html
8653 https://sunlight.net/catalog/pendants_272935.html
8654 https://sunlight.net/catalog/earring_320263.html
8655 htt

8788 https://sunlight.net/catalog/earring_79031.html
8789 https://sunlight.net/catalog/earring_269018.html
8790 https://sunlight.net/catalog/clock_315525.html
8791 https://sunlight.net/catalog/earring_88970.html
8792 https://sunlight.net/catalog/earring_244061.html
8793 https://sunlight.net/catalog/bracelets_253772.html
8794 https://sunlight.net/catalog/earring_92350.html
8795 https://sunlight.net/catalog/earring_245249.html
8796 https://sunlight.net/catalog/earring_242460.html
8797 https://sunlight.net/catalog/earring_89012.html
8798 https://sunlight.net/catalog/earring_98038.html
8799 https://sunlight.net/catalog/ring_259689.html
8800 https://sunlight.net/catalog/ring_309950.html
8801 https://sunlight.net/catalog/earring_276966.html
8802 https://sunlight.net/catalog/earring_92933.html
8803 https://sunlight.net/catalog/keychain_236586.html
8804 https://sunlight.net/catalog/earring_249577.html
8805 https://sunlight.net/catalog/bracelets_89502.html
8806 https://sunlight.net/catalog/neck

In [111]:
### Поиск ошибки
for i in range(4216, 4217):
    try:
        row = len(data)
        urlcard = domain +links[i]
        card = mysoup(urlcard)
        data.loc[row,'url'] = urlcard
        data.loc[row,'h1'] = card.find("h1").text.strip()
        data.loc[row,'art'] = card.find('div', class_="supreme-product-card__product-article-text").text.strip()
        data.loc[row,'price'] = card.find('div', class_="supreme-product-card__price-discount-price").text.strip().replace('\u202f', '').replace('\xa0', '')
        data.loc[row,'price2'] = card.find('div', class_="supreme-product-card__price-old").text.strip().replace('\u202f', '').replace('\xa0', '')
        a = card.find_all('p', class_="supreme-product-card__description supreme-product-card__description_default")
        data.loc[row,'gold'] = ';'.join([x.text for x in a])
        data.loc[row,'d1'] = "&".join([t.text for t in card.find_all('span', class_="supreme-product-card-description__item-text")])
        data.loc[row,'d0'] = "&".join([t.text for t in card.find_all('a', class_="supreme-product-card-description__item-text")])
        #print(i, urlcard)
    except (AttributeError,TypeError) as e:
        print('error',e)
    except IndexError:
        print('indexerror len of = ' , len(card.find_all('div','detail__item-option')) )
    print(i, urlcard)

4216 https://sunlight.net/catalog/earring_92815.html


In [367]:
len(data)

20

In [379]:
data

,h1,art,price,price2,gold,gold2,weight,gems,gems2,url,d1,d0
0,Часы женские,Артикул: 263766,990₽,,Артикул: 263766;Водонепроницаемость: 3 bar (30...,NaN,NaN,NaN,NaN,https://sunlight.net/catalog/clock_263766.html,V382ARN-01BM&Япония&Неометалл Zamak&IP&38 мм&Н...,SUN&Женщинам
1,Золотое обручальное кольцо,Артикул: 114165,11860₽,39533₽,Артикул: 114165;Кольцо \nБелое золото 585 п...,NaN,NaN,NaN,NaN,https://sunlight.net/catalog/ring_114165.html,"14-5260-12-00*&РОССИЯ&Белое золото&5,0 мм&3.04 г",SERGEY GRIBNYAKOV&Обручальные кольца
2,Серебряные серьги,Артикул: 273106,2490₽,4980₽,Артикул: 273106;Серьги \nСеребро 925 пробы....,NaN,NaN,NaN,NaN,https://sunlight.net/catalog/earring_273106.html,S16246-C9Y-01&ИТАЛИЯ&Серебро&20 мм&2.33 г,LINEA ITALIA&Серебро Италии
3,Серебряные серьги,Артикул: 88536,894₽,2980₽,Артикул: 88536;Серьги \nСеребро 925 пробы.;...,NaN,NaN,NaN,NaN,https://sunlight.net/catalog/earring_88536.html,S10622-C9W-01&Серебро&3.07 г,SERGEY GRIBNYAKOV&Гладкие серебряные украшения
4,Серебряный крест-подвеска,Артикул: 40983,690₽,2300₽,Артикул: 40983;Подвеска \nСеребро 925 пробы...,NaN,NaN,NaN,NaN,https://sunlight.net/catalog/pendants_40983.html,08331&РОССИЯ&Серебро&1.6 г,Елизавета&Христианство
...,...,...,...,...,...,...,...,...,...,...,...,...
551,Серебряная подвеска,Артикул: 124099,1390₽,4633₽,Артикул: 124099;Подвеска \nСеребро 925 проб...,NaN,NaN,NaN,NaN,https://sunlight.net/catalog/pendants_124099.html,П-20113&РОССИЯ&Серебро&3.96 г,SERGEY GRIBNYAKOV&Мусульманские украшения
552,Золотое кольцо,Артикул: 130384,14540₽,48467₽,Артикул: 130384;Кольцо с комфортной посадкой ...,NaN,NaN,NaN,NaN,https://sunlight.net/catalog/ring_130384.html,"00450-000-718*&Белое золото, Розовое золото&5,...",SERGEY GRIBNYAKOV&Обручальные кольца
553,Часы мужские,Артикул: 315527,6294₽,20980₽,Артикул: 315527;Механизм: Швейцария\nМатериал ...,NaN,NaN,NaN,NaN,https://sunlight.net/catalog/clock_315527.html,G4501SNS-21BS&Швейцария&Сапфировое&Нержавеющая...,SERGEY GRIBNYAKOV&Нержавеющая сталь 316L&Мужчинам
554,Платиновое обручальное кольцо,Артикул: 326431,19470₽,64900₽,Артикул: 326431;Кольцо \n Pt950 пробы.\n\nП...,NaN,NaN,NaN,NaN,https://sunlight.net/catalog/ring_326431.html,1-751-000П*&РОССИЯ&Платина&3.49 г,SERGEY GRIBNYAKOV&PLATINUM


In [18]:
domain+links[0]

'https://sunlight.net/catalog/ring_130995.html'

In [82]:
urlcard = domain +links[100]
card = mysoup(urlcard)


In [90]:
#collect 29-08-22
row = len(data)
data.loc[row,'url'] = urlcard
data.loc[row,'h1'] = card.find("h1").text.strip()
data.loc[row,'art'] = card.find('div', class_="supreme-product-card__product-article-text").text.strip()
data.loc[row,'price'] = card.find('div', class_="supreme-product-card__price-discount-price").text.strip().replace('\u202f', '').replace('\xa0', '')
data.loc[row,'price2'] = card.find('div', class_="supreme-product-card__price-old").text.strip().replace('\u202f', '').replace('\xa0', '')
a = card.find_all('p', class_="supreme-product-card__description supreme-product-card__description_default")
data.loc[row,'gold'] = ';'.join([x.text for x in a])
data.loc[row,'d1'] = "&".join([t.text for t in card.find_all('span', class_="supreme-product-card-description__item-text")])
data.loc[row,'d0'] = "&".join([t.text for t in card.find_all('a', class_="supreme-product-card-description__item-text")])

In [113]:
data

,h1,art,price,price2,gold,gold2,weight,gems,gems2,url,d1,d0
0,Золотая подвеска с бриллиантами,Артикул: 276709,13194₽,43980₽,"Артикул: 276709;Подвеска с 43 бриллиантами, ог...",NaN,NaN,NaN,NaN,https://sunlight.net/catalog/pendants_276709.html,71308-P5R-01&РОССИЯ&Розовое золото&Бриллиант&0...,Бриллианты Якутии&Бриллианты. Fashion Diamonds
1,Серебряное шейное украшение с фианитами,Артикул: 263399,1674₽,5580₽,"Артикул: 263399;Шейное укр., вставка: фианит;...",NaN,NaN,NaN,NaN,https://sunlight.net/catalog/neck_decoration_2...,S54895-H9W-01&Серебро&Фианит&3.0 г,SERGEY GRIBNYAKOV&СЕРЕБРО. Классика
2,Серебряные серьги с эмалью и фианитами,Артикул: 51951,1674₽,5580₽,"Артикул: 51951;Серьги, вставка: фианит; фиани...",NaN,NaN,NaN,NaN,https://sunlight.net/catalog/earring_51951.html,S2362-C9W-03&Серебро&Фианит; Эмаль&6.05 г,SERGEY GRIBNYAKOV&Эмаль Серебро
3,Серебряная подвеска с эмалью,Артикул: 88477,714₽,2380₽,"Артикул: 88477;Подвеска, вставка: эмаль; \...",NaN,NaN,NaN,NaN,https://sunlight.net/catalog/pendants_88477.html,15081.5&РОССИЯ&Серебро&Эмаль&1.16 г,SERGEY GRIBNYAKOV&HELLO KITTY
4,Серебряное кольцо с фианитами и топазами,Артикул: 121249,1314₽,4380₽,"Артикул: 121249;Кольцо, вставка: топаз; фиани...",NaN,NaN,NaN,NaN,https://sunlight.net/catalog/ring_121249.html,1021014428-1&РОССИЯ&Серебро&Фианит; Топаз&1.76 г,SERGEY GRIBNYAKOV&NATURIKA
...,...,...,...,...,...,...,...,...,...,...,...,...
5938,Золотые серьги,Артикул: 92933,7794₽,25980₽,Артикул: 92933;Серьги \nРозовое золото 585 ...,NaN,NaN,NaN,NaN,https://sunlight.net/catalog/earring_92933.html,Т100328607*&РОССИЯ&Розовое золото&1.29 г,SERGEY GRIBNYAKOV&Золотые украшения без вставок
5939,Серебряный браслет,Артикул: 89502,1790₽,5967₽,Артикул: 89502;Браслет \nСеребро 925 пробы....,NaN,NaN,NaN,NaN,https://sunlight.net/catalog/bracelets_89502.html,S10660-B9W-01&Серебро&6.77 г,SERGEY GRIBNYAKOV
5940,Серебряное шейное украшение с керамикой,Артикул: 82806,1314₽,4380₽,"Артикул: 82806;Шейное укр., вставка: керамика...",NaN,NaN,NaN,NaN,https://sunlight.net/catalog/neck_decoration_8...,S20089-H9M-01&Серебро&Керамика&4.48 г,SERGEY GRIBNYAKOV&СЕРЕБРО КЕРАМИКА
5941,Серебряное кольцо с фианитами и сапфирами,Артикул: 91007,1134₽,3780₽,"Артикул: 91007;Кольцо, вставка: фианит; нанос...",NaN,NaN,NaN,NaN,https://sunlight.net/catalog/ring_91007.html,64206Б.5&РОССИЯ&Серебро&Фианит; Сапфир&2.57 г,SERGEY GRIBNYAKOV&Драгоценные камни. Серебро


In [200]:
data.gold.loc[10]

'Артикул: 275835;Серьги, вставка:  фианит;    \nРозовое золото 585 пробы.\n\nКостромские ювелирные украшения: традиционный русский стиль -  Коллекция украшений с традиционными русскими дизайнами. Украшения несут в себе ювелирные традиции, любимые многими поколениями российских женщин, которые сохранят свою ценность на многие года.;Вес изделия: 2.19г;Серьги, вставка:  фианит;    \nРозовое золото 585 пробы.;Костромские ювелирные украшения: традиционный русский стиль -  Коллекция украшений с традиционными русскими дизайнами. Украшения несут в себе ювелирные традиции, любимые многими поколениями российских женщин, которые сохранят свою ценность на многие года.'

## Разработка rе

In [109]:
pattern_gold = r'[Сс]ереб\w+|[Зз]олот\w*|[Сс]таль'
pattern_diam = r'[Бб]риллиант\w*'
pattern_probe = r'[\d]{,3}(?=\s*проб)'
pattern_gramm = r'[\d.\d]{,5}(?=г;)'
pattern_vst = r'вставк\w*'#r'вставк\w*'
pattern_type = r'(?<=;)\w+\s?\w*'
def gold(t,patt):
    f = re.search(patt,t)
    return f.group() if f else 're не найдено'

In [421]:
t = 'Артикул: 114165;Кольцо    \nБелое золото 585 пробы.;Вес изделия: 3.04г;Кольцо    \nБелое золото 585 пробы.'
re.search(pattern_type,t)

<re.Match object; span=(16, 23), match='Кольцо '>

In [111]:
patt = pattern_type
#patt = pattern_diam

In [404]:
patt

'(?<=;)\\w+'

In [112]:
i = 1
gold(data.gold.loc[i],patt), data.gold.loc[i]

('Шейное укр',
 'Артикул: 209254;Шейное укр., вставка:  Сталь;    \n;Шейное укр., вставка:  Сталь;')

In [113]:
data.gold.apply(lambda x: gold(x, patt)).unique()

array(['Кольцо ', 'Шейное укр', 'Подвеска ', 'Браслет ', 'Серьги ',
       'Функции', 'Браслет', 'Колье', 'Цепь ручной', 'Гарантийный срок',
       'Брелок', 'Вставка для', 'Серьги', 'Серьга ', 'Брошь', 'Цепь ',
       'Брошь ', 'Для кого', 'Кольцо с', 'Запонки', 'Зажим для',
       'Форма корпуса', 'Пирсинг ', 'Кольцо', 'Ножной браслет',
       'Браслет ручной', 'Серьга', 'Ложка детская', 'Механизм',
       'Материал браслета', 'Запонки ', 'Материал корпуса', 'Ручка ',
       'Ложка кофейная', 'Серьги SOKOLOV', 'Колье ', 'Диаметр корпуса',
       'Шнурок ', 'Водонепроницаемость', 'Брелок ', 'Сувенир ',
       'Подвеска', 'Кулон ', 'Набор крестильный', 'Булавка ',
       'Ионизатор ', 'Пирсинг', 'Чайный набор', 'Шейное украшение',
       'Кольцо на', 'Кольцо SOKOLOV', 'Ложка ', 'Шнурок', 'Подвеска qq',
       'Гайтан', 'Икона ', 'Кольцо о', 'Средство SUNLIGHT', 'Монета ',
       'Серьги со', '4', 'Украшение для', 'Колье SOKOLOV',
       'Браслет золото', 'Мощевик ', 'Подвеска SOKOLOV',

In [427]:
data.gold.apply(lambda x: gold(x, patt)).value_counts()

Серьги                 201
Кольцо                  70
Браслет                 65
Подвеска                58
Шейное укр              51
Функции                 17
Гарантийный срок         9
Кольцо                   8
Браслет                  7
Серьги                   7
Диаметр корпуса          6
Водонепроницаемость      6
Колье                    5
Серьга                   5
Брошь                    4
Механизм                 4
Ножной браслет           3
Пирсинг                  3
Цепь                     3
Значок                   2
Кольцо с                 2
Сувенир                  2
Материал браслета        2
Запонки                  2
Материал корпуса         2
Браслет ручной           2
Серьга                   2
Серебряный сувенир       1
Булавка                  1
Форма корпуса            1
Набор                    1
Кулон                    1
Цепь ручной              1
Подвеска                 1
Для кого                 1
Name: gold, dtype: int64

In [424]:
data[data.gold.apply(lambda x: gold(x, patt))=='Ножной']

,h1,art,price,price2,gold,gold2,weight,gems,gems2,url,d1,d0,diam,goldsilv,probe,gramm,vst
50,Золотой ножной браслет,Артикул: 277839,5690₽,18967₽,Артикул: 277839;Ножной браслет \nРозовое зо...,NaN,NaN,NaN,NaN,https://sunlight.net/catalog/anklets_277839.html,12655-A5R-01&Розовое золото&1.11 г,JASMİNE JEWELLERY&ЗОЛОТО ИТАЛИИ,re не найдено,Золотой,585,1.11,re не найдено
374,Золотой ножной браслет,Артикул: 277838,6330₽,21100₽,Артикул: 277838;Ножной браслет \nРозовое зо...,NaN,NaN,NaN,NaN,https://sunlight.net/catalog/anklets_277838.html,12654-A5R-01&Розовое золото&1.11 г,JASMİNE JEWELLERY&ЗОЛОТО ИТАЛИИ,re не найдено,Золотой,585,1.11,re не найдено
495,Золотой ножной браслет,Артикул: 277844,6670₽,22233₽,Артикул: 277844;Ножной браслет \nРозовое зо...,NaN,NaN,NaN,NaN,https://sunlight.net/catalog/anklets_277844.html,12659-A5R-01&Розовое золото&1.14 г,JASMİNE JEWELLERY&ЗОЛОТО ИТАЛИИ,re не найдено,Золотой,585,1.14,re не найдено


## Результат первичной обработки 

In [114]:
#res
data['diam'] = data.gold.apply(lambda x: gold(x, pattern_diam))

data['goldsilv'] = data.h1.apply(lambda x: gold(x, pattern_gold))

data['probe'] = data.gold.apply(lambda x: gold(x, pattern_probe))

data['gramm'] = data.gold.apply(lambda x: gold(x, pattern_gramm))

data['vst'] = data.gold.apply(lambda x: gold(x, pattern_vst))

data['type'] = data.gold.apply(lambda x: gold(x, pattern_type))

In [ ]:
#разработка

In [ ]:
<div class="supreme-product-card-description__item-title">
    Бренд
  </div>

In [47]:
card.find_all('span', class_="supreme-product-card-description__item-text")[4].text

'1.06 г'

In [ ]:
<a href="https://sunlight.net/catalog/sergey-gribnyakov.html" class="supreme-product-card-description__item-text">SERGEY GRIBNYAKOV</a>

In [50]:
card.find_all('a', class_="supreme-product-card-description__item-text")

[<a class="supreme-product-card-description__item-text" href="https://sunlight.net/catalog/sergey-gribnyakov.html">SERGEY GRIBNYAKOV</a>,
 <a class="supreme-product-card-description__item-text" href="https://sunlight.net/catalog/?collection=6499">ЗОЛОТО. ТОПАЗЫ</a>]

In [ ]:
<div class="supreme-product-card__product-article-text">
      Артикул: 165495
    </div>

In [80]:
card.find('div', class_="supreme-product-card__product-article-text").text.strip()

'Артикул: 165495'

In [ ]:
<div class="supreme-product-card__price-discount-price">
  5 395&nbsp;₽
</div>

In [95]:
card.find('div', class_="supreme-product-card__price-old").text.strip().replace('\u202f', '').replace('\xa0', '')

'26975₽'

In [89]:
card.find('div', class_="supreme-product-card__price-discount-price")

<div class="supreme-product-card__price-discount-price">
  5 395 ₽
</div>

In [ ]:
<div class="supreme-product-card__price-old">
  26 975&nbsp;₽
</div>

In [ ]:
<p class="supreme-product-card__description supreme-product-card__description_default">Кольцо «Бриллианты Якутии» с 21 бриллиантом, огранка круг 17 граней, 0.04 карат, цвет 2, чистота 2, тип огранки Б; 
Розовое золото 585 пробы</p>

In [101]:
a = card.find_all('p', class_="supreme-product-card__description supreme-product-card__description_default")

In [105]:
a = card.find_all('p', class_="supreme-product-card__description supreme-product-card__description_default")
';'.join([x.text for x in a])

'Артикул: 165495;Кольцо «Бриллианты Якутии» с 21 бриллиантом, огранка круг 17 граней, 0.04 карат, цвет 2, чистота 2, тип огранки Б; \nРозовое золото 585 пробы;Вес изделия: 0.52г;Кольцо «Бриллианты Якутии» с 21 бриллиантом, огранка круг 17 граней, 0.04 карат, цвет 2, чистота 2, тип огранки Б; \nРозовое золото 585 пробы'

## Визуализация

In [93]:
 urlcard = domain +links[i]
        card = mysoup(urlcard)
        row = len(data)
        data.loc[row,'url'] = urlcard
        data.loc[row,'h1'] = card.find("h1").text.strip()
        data.loc[row,'art'] = card.select(
            'body > div.grid > div > div > div.shops__breadcrumbs.catalog_breadcrumbs > div > div:nth-child(4) > span > span')[0].text
        data.loc[row,'price'] = float(''.join(re.findall(r'\d+', card.find(
            "div","places-cont__price b-price").find("div", 'new').text)))
        
        data.loc[row,'price2'] = float(''.join(re.findall(r'\d+', card.find('div',"detail__item-price-new").text)))
        data.loc[row,'gold'] = card.find_all('span','href')[2].text.strip()
        data.loc[row,'gold2'] = re.findall(r"(Золото\s+\d+)",str(card))[0]
        data.loc[row,'weight'] = float(re.findall(r"(?:Вес:</span>\n)\s+(\d+.\d+)",str(card))[0])
        div_gems = card.select('body > div.grid > div > div > div.detail__item.detail__item--new > div.detail__item-col._3.position-relative > form > div:nth-child(8)')
        data.loc[row,'gems'] = re.findall(r'Камни:\s+([\w\W]+)', div_gems[0].text)
        div_gems = card.select('body > div.grid > div > div > div.detail__item.detail__item--new > div.detail__item-col._3.position-relative > form > div:nth-child(8)')
        data.loc[row,'gems2'] = re.search(r'Камни:\s+([\w\W]+)', str(div_gems[0].text.strip()))[1]

IndentationError: unexpected indent (<ipython-input-93-acfefae476ee>, line 2)

In [68]:
card.find_all('span','href')[2].text.strip()

IndexError: list index out of range

In [43]:
div_gems = card.select('body > div.grid > div > div > div.detail__item.detail__item--new > div.detail__item-col._3.position-relative > form > div:nth-child(8)')
#data.loc[row,'gems'] = 
re.findall(r'Камни:\s+([\w\W]+)', div_gems[0].text)

['1  Бриллиант огранки "Круглая" граней - 57, тип огранки A, цвет 3, чистота 6, 0.09 карат; ']

In [64]:
#25-08-2022 all
#06-07-2022 серебро
#16-06-2022 Для Алексея Сапфир Изумруд Рубин
#24-05-2022
#14-04-2022

for i in range(len(links)):
    try:
        urlcard = domain +links[i]
        card = mysoup(urlcard)
        row = len(data)
        data.loc[row,'url'] = urlcard
        data.loc[row,'h1'] = card.find("h1").text.strip()
        data.loc[row,'art'] = card.select(
            'body > div.grid > div > div > div.shops__breadcrumbs.catalog_breadcrumbs > div > div:nth-child(4) > span > span')[0].text
        data.loc[row,'price'] = float(''.join(re.findall(r'\d+', card.find(
            "div","places-cont__price b-price").find("div", 'new').text)))
        
        data.loc[row,'price2'] = float(''.join(re.findall(r'\d+', card.find('div',"detail__item-price-new").text)))
        data.loc[row,'gold'] = card.find_all('span','href')[2].text.strip()
        data.loc[row,'gold2'] = re.findall(r"(Золото\s+\d+)",str(card))[0]
        data.loc[row,'weight'] = float(re.findall(r"(?:Вес:</span>\n)\s+(\d+.\d+)",str(card))[0])
        div_gems = card.select('body > div.grid > div > div > div.detail__item.detail__item--new > div.detail__item-col._3.position-relative > form > div:nth-child(8)')
        data.loc[row,'gems'] = re.findall(r'Камни:\s+([\w\W]+)', div_gems[0].text)
        div_gems = card.select('body > div.grid > div > div > div.detail__item.detail__item--new > div.detail__item-col._3.position-relative > form > div:nth-child(8)')
        data.loc[row,'gems2'] = re.search(r'Камни:\s+([\w\W]+)', str(div_gems[0].text.strip()))[1]
        
    except (AttributeError,TypeError) as e:
        print('error',e)
    except IndexError:
        print('indexerror len of = ' , len(card.find_all('div','detail__item-option')) )
    print(i, urlcard)


0 https://miuz.ru/catalog/rings/R01-PL-33915/
1 https://miuz.ru/catalog/earrings/E01-34598-BT/
2 https://miuz.ru/catalog/earrings/E01-RS-34112-EM/
3 https://miuz.ru/catalog/earrings/E4188-E140-CZ-ZI/
4 https://miuz.ru/catalog/pendants/P2042-LS-0053-R/
5 https://miuz.ru/catalog/earrings/E774-007808490004/
error 'NoneType' object is not subscriptable
6 https://miuz.ru/catalog/chain/C30-NC12-002-035/
7 https://miuz.ru/catalog/rings/R4150-D-LRP34737AM/
indexerror len of =  4
8 https://miuz.ru/catalog/rings/R4150-S-R906A13124TR/
9 https://miuz.ru/catalog/rings/R01-33775/
indexerror len of =  0
10 https://miuz.ru/catalog/cufflinks/K108-1982155BMD/
indexerror len of =  4
11 https://miuz.ru/catalog/rings/R2036-KL-4953/
12 https://miuz.ru/catalog/rings/R2700-IGR-16493TM/
13 https://miuz.ru/catalog/rings/R01-33636-SA/
14 https://miuz.ru/catalog/earrings/E01-L-PL-34418/
15 https://miuz.ru/catalog/pendants/P2020-095175-339522/
16 https://miuz.ru/catalog/earrings/E2028-S132-415M1/
17 https://miuz.r

133 https://miuz.ru/catalog/pendants/P01-34021-PR/
134 https://miuz.ru/catalog/necklace/N157-KJN117/
135 https://miuz.ru/catalog/earrings/E01-L-35553-GR/
136 https://miuz.ru/catalog/earrings/E01-Y-60382-Z/
indexerror len of =  5
137 https://miuz.ru/catalog/earrings/E01-SDI-35195-35/
error 'NoneType' object is not subscriptable
138 https://miuz.ru/catalog/earrings/E37-T100022325/
139 https://miuz.ru/catalog/rings/R01-35837/
140 https://miuz.ru/catalog/rings/R01-L-34760/
indexerror len of =  4
141 https://miuz.ru/catalog/necklace/N2036-PR-2598/
142 https://miuz.ru/catalog/rings/R01-SOL74-040-G3/
143 https://miuz.ru/catalog/pendants/P2028-P130-338/
144 https://miuz.ru/catalog/earrings/E01-35401-SA/
145 https://miuz.ru/catalog/rings/R01-34446-SA/
146 https://miuz.ru/catalog/brooches/H01-35181-PR/
147 https://miuz.ru/catalog/earrings/E01-34056-EM/
148 https://miuz.ru/catalog/earrings/E01-L-35022-MIX/
149 https://miuz.ru/catalog/pendants/P01-BS-0134/
150 https://miuz.ru/catalog/earrings/E01-

272 https://miuz.ru/catalog/rings/R4150-C-LR636E/
273 https://miuz.ru/catalog/rings/R01-CL-33839-C22/
indexerror len of =  4
274 https://miuz.ru/catalog/rings/M2029-1100415061-EL/
275 https://miuz.ru/catalog/pendants/P2020-096994-1-PM/
276 https://miuz.ru/catalog/necklace/N2018-PC010027ASA/
277 https://miuz.ru/catalog/earrings/E01-PL-34308/
278 https://miuz.ru/catalog/rings/R2018-RR03036ASA-R17/
279 https://miuz.ru/catalog/rings/R178-R-6/
280 https://miuz.ru/catalog/rings/R01-SOL38-025-G2/
281 https://miuz.ru/catalog/rings/R01-L-PL-35552-B/
282 https://miuz.ru/catalog/earrings/E37-T142028332-Z/
283 https://miuz.ru/catalog/piercing/I01-60009-Z/
indexerror len of =  3
284 https://miuz.ru/catalog/earrings/E4150-SIL21064/
indexerror len of =  0
285 https://miuz.ru/catalog/necklace/N77-N49336-SA-R17/
286 https://miuz.ru/catalog/earrings/E2018-ER230126ADI/
287 https://miuz.ru/catalog/rings/R2703-RG37567-R17/
288 https://miuz.ru/catalog/earrings/E01-59788-Z/
289 https://miuz.ru/catalog/earrin

411 https://miuz.ru/catalog/rings/R01-L-PL-35572/
412 https://miuz.ru/catalog/rings/R01-46084-SA/
413 https://miuz.ru/catalog/earrings/E157-TWE030/
414 https://miuz.ru/catalog/earrings/E01-1851352BD/
error 'NoneType' object is not subscriptable
415 https://miuz.ru/catalog/pendants/P2040-XP211141-DC/
416 https://miuz.ru/catalog/rings/R4150-D-48458SA/
417 https://miuz.ru/catalog/rings/R01-35468/
418 https://miuz.ru/catalog/rings/R2009-1982997AXD-R17/
419 https://miuz.ru/catalog/rings/R01-34267/
420 https://miuz.ru/catalog/earrings/E01-L-35019-SC/
421 https://miuz.ru/catalog/pendants/P178-IGP-9899-0.10/
422 https://miuz.ru/catalog/earrings/E01-FSW-59784-B-5/
423 https://miuz.ru/catalog/rings/R01-PL-34272/
424 https://miuz.ru/catalog/rings/R01-SS-35598-B/
425 https://miuz.ru/catalog/rings/R4172-48457-EM/
426 https://miuz.ru/catalog/rings/R01-PL-33916/
427 https://miuz.ru/catalog/rings/R755-46021R242/
428 https://miuz.ru/catalog/pendants/P178-IGP-12760/
429 https://miuz.ru/catalog/earrings/

error 'NoneType' object is not subscriptable
554 https://miuz.ru/catalog/rings/R01-VIT-0075/
error 'NoneType' object is not subscriptable
555 https://miuz.ru/catalog/necklace/N2042-LIO-0020/
556 https://miuz.ru/catalog/pendants/P01-35812MIX/
557 https://miuz.ru/catalog/earrings/E163-S1326107-ZI/
558 https://miuz.ru/catalog/earrings/E2018-ER01076ADI/
559 https://miuz.ru/catalog/bracelets/B157-TTB002-H/
560 https://miuz.ru/catalog/earrings/E01-34159-SA/
561 https://miuz.ru/catalog/earrings/E2020-40292417-BT/
562 https://miuz.ru/catalog/earrings/E2018-ER020067AWP/
563 https://miuz.ru/catalog/earrings/E01-L-35562-B/
564 https://miuz.ru/catalog/rings/R4150-C-LR1749E/
565 https://miuz.ru/catalog/earrings/E01-34075-AQ/
566 https://miuz.ru/catalog/rings/R2018-RRU1074AEM-R17/
indexerror len of =  4
567 https://miuz.ru/catalog/pendants/P01-SILVER-019-SA/
568 https://miuz.ru/catalog/rings/R4150-D-LRP33085S/
569 https://miuz.ru/catalog/rings/R01-60417-Z/
570 https://miuz.ru/catalog/earrings/E2001-

...

error 'NoneType' object is not subscriptable
4365 https://miuz.ru/catalog/chain/C2000-040247060/
indexerror len of =  4
4366 https://miuz.ru/catalog/clock/W773-RM6159TL1RMC6R/
4367 https://miuz.ru/catalog/rings/R01-L-PL-35042/
4368 https://miuz.ru/catalog/earrings/E755-70640E002-R17/
4369 https://miuz.ru/catalog/pendants/P2020-104000392493/
4370 https://miuz.ru/catalog/rings/R2022-SA2472RTN/
4371 https://miuz.ru/catalog/earrings/E01-35277-BT/
4372 https://miuz.ru/catalog/earrings/E2017-E304261SAP-R17/
error 'NoneType' object has no attribute 'text'
4373 https://miuz.ru/catalog/clock/W133-0551.2.9.58/
4374 https://miuz.ru/catalog/rings/R01-SOL38-015-G1/
4375 https://miuz.ru/catalog/earrings/E4199-EP-00777/
4376 https://miuz.ru/catalog/earrings/E4150-D-ERGM12974OX/
4377 https://miuz.ru/catalog/earrings/E01-R-33839-GR/
4378 https://miuz.ru/catalog/pendants/P01-34368-EM/
error 'NoneType' object is not subscriptable
4379 https://miuz.ru/catalog/necklace/N2042-ODR-0043/
4380 https://miuz.ru/

4493 https://miuz.ru/catalog/rings/R127-UFOQ5854G/
error 'NoneType' object is not subscriptable
4494 https://miuz.ru/catalog/crosses/J37-T13006576/
4495 https://miuz.ru/catalog/rings/R01-SP35-025/
4496 https://miuz.ru/catalog/earrings/E2020-104000292942/
4497 https://miuz.ru/catalog/earrings/E755-70654E001-R17/
4498 https://miuz.ru/catalog/earrings/E01-L-PL-35543-B/
4499 https://miuz.ru/catalog/earrings/E129-EO078499/
indexerror len of =  4
4500 https://miuz.ru/catalog/rings/R110-01K1511420/
4501 https://miuz.ru/catalog/rings/R01-L-35423-RO/
4502 https://miuz.ru/catalog/rings/R97-MR13362/
4503 https://miuz.ru/catalog/earrings/E01-59538-Z-Y/
error 'NoneType' object is not subscriptable
4504 https://miuz.ru/catalog/pendants/P37-T140635216/
error 'NoneType' object is not subscriptable
4505 https://miuz.ru/catalog/necklace/N2042-LIO-0024/
4506 https://miuz.ru/catalog/rings/R77-R47772-R17/
4507 https://miuz.ru/catalog/earrings/E01-35275/
4508 https://miuz.ru/catalog/earrings/E01-35701-B/
45

error 'NoneType' object is not subscriptable
4622 https://miuz.ru/catalog/necklace/N2040-LIO-0013/
4623 https://miuz.ru/catalog/rings/R2001-MIS-0008-R/
4624 https://miuz.ru/catalog/rings/R4150-D-LRP31835/
4625 https://miuz.ru/catalog/rings/R01-SOL38-025-G1/
4626 https://miuz.ru/catalog/pendants/P37-T102035221-ZI/
indexerror len of =  3
4627 https://miuz.ru/catalog/rings/R2029-1000014616/
4628 https://miuz.ru/catalog/rings/R2021-012-0761-BT/
4629 https://miuz.ru/catalog/earrings/E2022-SA2484ESABL/
4630 https://miuz.ru/catalog/rings/R108-BRN-1981444-R17/
4631 https://miuz.ru/catalog/rings/R01-34515-SS/
4632 https://miuz.ru/catalog/pendants/P01-33679/
4633 https://miuz.ru/catalog/earrings/E2018-1983053CP-R17/
4634 https://miuz.ru/catalog/pendants/P01-59545-Z/
4635 https://miuz.ru/catalog/earrings/E01-L-PL-35313-B/
4636 https://miuz.ru/catalog/earrings/E4211-ER2388WDI1/
4637 https://miuz.ru/catalog/rings/R175-R30220/
4638 https://miuz.ru/catalog/earrings/E774-002252780008/
4639 https://miu

4755 https://miuz.ru/catalog/rings/R4150-C-6101S/
4756 https://miuz.ru/catalog/bracelets/B01-34955-66/
4757 https://miuz.ru/catalog/earrings/E01-35714-BSA/
4758 https://miuz.ru/catalog/earrings/E2022-E052680/
4759 https://miuz.ru/catalog/earrings/E01-57538-BT/
4760 https://miuz.ru/catalog/earrings/E01-35112-PR/
4761 https://miuz.ru/catalog/earrings/E01-MLN162-010/
4762 https://miuz.ru/catalog/pendants/P108-198605DCMXX-R17/
error 'NoneType' object is not subscriptable
4763 https://miuz.ru/catalog/earrings/E2041-MIN-0005/
indexerror len of =  4
4764 https://miuz.ru/catalog/earrings/E01-SILVER-025-6.0-Z/
4765 https://miuz.ru/catalog/pendants/P163-Y-P43211868/
4766 https://miuz.ru/catalog/rings/R755-R35939-R17/
4767 https://miuz.ru/catalog/rings/R4172-45196-SA/
indexerror len of =  4
4768 https://miuz.ru/catalog/chain/C191-810551414/
error 'NoneType' object is not subscriptable
4769 https://miuz.ru/catalog/chain/C191-110301417/
4770 https://miuz.ru/catalog/rings/R2022-SA2019R/
4771 https:/

4886 https://miuz.ru/catalog/rings/R01-MIS-0064/
error 'NoneType' object is not subscriptable
4887 https://miuz.ru/catalog/bracelets/B01-ARB-60323/
4888 https://miuz.ru/catalog/earrings/E01-L-PL-35014/
error 'NoneType' object is not subscriptable
4889 https://miuz.ru/catalog/pendants/P01-BON48582-W-070/
4890 https://miuz.ru/catalog/rings/R01-WED-00053/
4891 https://miuz.ru/catalog/earrings/E01-34105/
indexerror len of =  4
4892 https://miuz.ru/catalog/rings/R4150-S-R906A13124W/
4893 https://miuz.ru/catalog/bracelets/B157-HB210/
4894 https://miuz.ru/catalog/earrings/E2028-S132-835/
4895 https://miuz.ru/catalog/pendants/P01-34090-LB/
4896 https://miuz.ru/catalog/pendants/P01-L-35072-SC/
4897 https://miuz.ru/catalog/earrings/E01-ICE-35908/
4898 https://miuz.ru/catalog/rings/R4150-C-6102E-CIN/
4899 https://miuz.ru/catalog/rings/R01-34368-SA/
4900 https://miuz.ru/catalog/earrings/E01-L-35421-B/
4901 https://miuz.ru/catalog/earrings/E01-RS-34208-EM/
4902 https://miuz.ru/catalog/earrings/E01-

In [65]:
data

,h1,art,price,price2,gold,gold2,weight,gems,gems2,url
0,Кольцо c бриллиантомR01-PL-33915,Артикул R01-PL-33915,31995.0,30495.0,Как измерить размер?,Золото 585,1.56,"[1 Бриллиант огранки ""Круглая"" граней - 57, т...","1 Бриллиант огранки ""Круглая"" граней - 57, ти...",https://miuz.ru/catalog/rings/R01-PL-33915/
1,Серьги c топазами и фианитамиE01-34598-BT,Артикул E01-34598-BT,13280.0,13035.0,Золото 585,Золото 585,2.13,"[12 Фианитов огранки ""Круглая"", 0.08 карат;2 ...","12 Фианитов огранки ""Круглая"", 0.08 карат;2 ...",https://miuz.ru/catalog/earrings/E01-34598-BT/
2,"Серьги c бриллиантами, изумрудами и цветными с...",Артикул E01-RS-34112-EM,145197.0,136197.0,Золото 585,Золото 585,7.67,"[102 Бриллианта огранки ""Круглая"" граней - 57...","102 Бриллианта огранки ""Круглая"" граней - 57,...",https://miuz.ru/catalog/earrings/E01-RS-34112-EM/
3,Серьги c фианитамиE4188-E140-CZ-ZI,Артикул E4188-E140-CZ-ZI,12077.0,12077.0,Золото 585,Золото 585,1.65,"[2 Фианита, 0.824 карат; ]","2 Фианита, 0.824 карат;",https://miuz.ru/catalog/earrings/E4188-E140-CZ...
4,Подвеска c фианитомP2042-LS-0053-R,Артикул P2042-LS-0053-R,4026.0,4026.0,Золото 585,Золото 585,0.55,"[1 Фианит огранки ""Круглая"", 0.09 карат; ]","1 Фианит огранки ""Круглая"", 0.09 карат;",https://miuz.ru/catalog/pendants/P2042-LS-0053-R/
...,...,...,...,...,...,...,...,...,...,...
4965,Серьги c топазами и лондон топазомE01-59823-MIX,Артикул E01-59823-MIX,32556.0,30108.0,Золото 585,Золото 585,4.92,"[18 Лондон Топазов огранки ""Груша"", 3.73 кара...","18 Лондон Топазов огранки ""Груша"", 3.73 карат...",https://miuz.ru/catalog/earrings/E01-59823-MIX/
4966,Кольцо c бриллиантомR01-SOL32-030-G3,Артикул R01-SOL32-030-G3,96995.0,77495.0,Как измерить размер?,Золото 585,2.68,"[1 Бриллиант огранки ""Круглая"" граней - 57, т...","1 Бриллиант огранки ""Круглая"" граней - 57, ти...",https://miuz.ru/catalog/rings/R01-SOL32-030-G3/
4967,Подвеска c бриллиантами и цветными сапфирамиP0...,Артикул P01-EX-52862-SA-B,199245.0,192995.0,Золото 585,Золото 585,4.08,"[24 Бриллианта огранки ""Круглая"" граней - 57,...","24 Бриллианта огранки ""Круглая"" граней - 57, ...",https://miuz.ru/catalog/pendants/P01-EX-52862-...
4968,Подвеска c цветными сапфирамиP01-L-35018-SC,Артикул P01-L-35018-SC,15995.0,15745.0,Золото 585,Золото 585,1.34,"[12 Цветных сапфиров, Круглая огранка, цвет Ч...","12 Цветных сапфиров, Круглая огранка, цвет Че...",https://miuz.ru/catalog/pendants/P01-L-35018-SC/


In [66]:
data.to_excel('pars_16-06-2022_muiz_RSE.xlsx')

### Версия 14-04-2022

In [103]:
data = pd.DataFrame(columns=['h1','art','price','gold','weight','gems','url'])

In [16]:
#16-06-2022 Для Алексея Сапфир Изумруд Рубин
#24-05-2022
#14-04-2022

for i in range(len(links)):
    try:
        urlcard = domain +links[i]
        card = mysoup(urlcard)
        row = len(data)
        data.loc[row,'url'] = urlcard
        data.loc[row,'h1'] = card.find("h1").text.strip()
        data.loc[row,'art'] = card.select(
            'body > div.grid > div > div > div.shops__breadcrumbs.catalog_breadcrumbs > div > div:nth-child(4) > span > span')[0].text
        data.loc[row,'price'] = float(''.join(re.findall(r'\d+', card.find(
            "div","places-cont__price b-price").find("div", 'new').text)))
        
        #data.loc[row,'price'] = float(''.join(re.findall(r'\d+', card.find('div',"detail__item-price-new").text)))
        data.loc[row,'gold'] = re.findall(r"(Золото\s+\d+)",str(card))[0]
        data.loc[row,'weight'] = float(re.findall(r"(?:Вес:</span>\n)\s+(\d+.\d+)",str(card))[0])
        div_gems = card.select('body > div.grid > div > div > div.detail__item.detail__item--new > div.detail__item-col._3.position-relative > form > div:nth-child(8)')
        data.loc[row,'gems'] = re.search(r'Камни:\s+([\w\W]+)', str(div_gems[0].text.strip()))[1]
    except (AttributeError,TypeError) as e:
        print('error',e)
    except IndexError:
        print('indexerror len of = ' , len(card.find_all('div','detail__item-option')) )
    print(i, urlcard)


0 https://miuz.ru/catalog/rings/R01-35815EM/
1 https://miuz.ru/catalog/rings/R01-EX-52694-EM/
2 https://miuz.ru/catalog/pendants/P01-33599-SA/
3 https://miuz.ru/catalog/earrings/E01-SS-35483-EM/
4 https://miuz.ru/catalog/rings/R01-34095-SA/
5 https://miuz.ru/catalog/rings/R01-L-35098-RO/
6 https://miuz.ru/catalog/pendants/P01-L-35041-RO/
7 https://miuz.ru/catalog/rings/R108-135248AQQ65SR/
8 https://miuz.ru/catalog/earrings/E01-34419-SA/
9 https://miuz.ru/catalog/earrings/E01-34511-SA/
10 https://miuz.ru/catalog/earrings/E2017-E312786SAP/
11 https://miuz.ru/catalog/earrings/E01-34579-EM/
12 https://miuz.ru/catalog/necklace/N108-FST-0012/
13 https://miuz.ru/catalog/rings/R108-FST-0023/
14 https://miuz.ru/catalog/earrings/E755-69005E002-R17/
15 https://miuz.ru/catalog/rings/R108-FST-0010/
16 https://miuz.ru/catalog/earrings/E01-34449-EM/
17 https://miuz.ru/catalog/rings/R01-34447-SA/
18 https://miuz.ru/catalog/rings/R01-34384-EM/
19 https://miuz.ru/catalog/rings/R01-33860-EM/
20 https://m

151 https://miuz.ru/catalog/crosses/J01-33589-EM/
152 https://miuz.ru/catalog/rings/R01-L-34846-EM/
153 https://miuz.ru/catalog/rings/R97-MR18101-SA/
154 https://miuz.ru/catalog/rings/R77-R58878-SA/
155 https://miuz.ru/catalog/rings/R755-63883R014/
156 https://miuz.ru/catalog/rings/R4150-C-LRT23845S/
157 https://miuz.ru/catalog/rings/R4145-56579SA/
error 'NoneType' object is not subscriptable
158 https://miuz.ru/catalog/rings/R4150-C-LR789E-CIN/
159 https://miuz.ru/catalog/rings/R4150-C-LR727S/
indexerror len of =  0
160 https://miuz.ru/catalog/rings/R2022-SA2477RRU/
161 https://miuz.ru/catalog/rings/R2018-RRU1082AEM-R17/
162 https://miuz.ru/catalog/rings/R2018-RRU1070ASA-R17/
163 https://miuz.ru/catalog/rings/R2017-R300860EMR-R17/
164 https://miuz.ru/catalog/rings/R131-R4383-SA/
165 https://miuz.ru/catalog/rings/R108-351680ASE-R17/
166 https://miuz.ru/catalog/rings/R01-SS-35436-EM/
167 https://miuz.ru/catalog/rings/R01-SS-35505-EM/
168 https://miuz.ru/catalog/rings/R01-L-35068-MIX/
16

307 https://miuz.ru/catalog/rings/R01-35401-SA/
308 https://miuz.ru/catalog/rings/R01-34368-SA/
indexerror len of =  4
309 https://miuz.ru/catalog/earrings/E2018-ER26026ASA-R17/
310 https://miuz.ru/catalog/earrings/E01-34035-EM/
311 https://miuz.ru/catalog/earrings/E01-L-34846-EM/
312 https://miuz.ru/catalog/earrings/E2017-E312798EMR/
313 https://miuz.ru/catalog/earrings/E01-34115-EM/
indexerror len of =  4
314 https://miuz.ru/catalog/pendants/P184-SP19894-OS/
315 https://miuz.ru/catalog/earrings/E108-198794CS-R17/
316 https://miuz.ru/catalog/earrings/E01-L-35336-EM/
317 https://miuz.ru/catalog/earrings/E01-33502-SA/
318 https://miuz.ru/catalog/rings/R01-BS-0054-SA/
319 https://miuz.ru/catalog/rings/R2017-R306177EMP-R17/
320 https://miuz.ru/catalog/rings/R01-34239-EM/
321 https://miuz.ru/catalog/bracelets/B157-MOSB008/
322 https://miuz.ru/catalog/rings/R01-34020-SA/
323 https://miuz.ru/catalog/earrings/E01-33748-EM/
indexerror len of =  0
324 https://miuz.ru/catalog/earrings/E01-18317-

459 https://miuz.ru/catalog/rings/R01-EX-52840-EM/
460 https://miuz.ru/catalog/rings/R01-35752-EM/
461 https://miuz.ru/catalog/rings/R108-93259AR/
462 https://miuz.ru/catalog/rings/R01-34426-SA/
463 https://miuz.ru/catalog/rings/R01-34383-EM/
464 https://miuz.ru/catalog/rings/R01-34115-RO/
indexerror len of =  4
465 https://miuz.ru/catalog/pendants/P01-SILVER-006-EM/
indexerror len of =  4
466 https://miuz.ru/catalog/pendants/P01-SILVER-006-SA/
467 https://miuz.ru/catalog/pendants/P01-46522-SA/
468 https://miuz.ru/catalog/pendants/P01-34368-SA/
469 https://miuz.ru/catalog/pendants/P01-33592-RU/
470 https://miuz.ru/catalog/brooches/H2031-740216-MIX/
471 https://miuz.ru/catalog/brooches/H2031-740246-MIX/
472 https://miuz.ru/catalog/earrings/E755-59623E-003/
473 https://miuz.ru/catalog/earrings/E4155-C-022-0498SA/
474 https://miuz.ru/catalog/earrings/E4150-C-ER288E-CIN/
475 https://miuz.ru/catalog/earrings/E4150-C-49353S-CIN/
476 https://miuz.ru/catalog/earrings/E4150-C-49313S-CIN/
477 ht

600 https://miuz.ru/catalog/earrings/E01-34416-SA/
indexerror len of =  5
601 https://miuz.ru/catalog/earrings/E01-34230-SA/
602 https://miuz.ru/catalog/rings/R97-MR11996-SA/
603 https://miuz.ru/catalog/rings/R4150-D-1981674RR17/
604 https://miuz.ru/catalog/pendants/P01-34597-SA/
indexerror len of =  4
605 https://miuz.ru/catalog/brooches/H01-EX-52892-RU/
606 https://miuz.ru/catalog/rings/R2017-R305596SAP-R17/
indexerror len of =  4
607 https://miuz.ru/catalog/rings/R4150-S-LRP35184E/
indexerror len of =  4
608 https://miuz.ru/catalog/rings/R4150-S-LRP34571E/
indexerror len of =  4
609 https://miuz.ru/catalog/rings/R4150-S-LRP34574S/
indexerror len of =  4
610 https://miuz.ru/catalog/rings/R4150-S-LRG82022S/
indexerror len of =  4
611 https://miuz.ru/catalog/rings/R4150-S-LRCJ3317SR17/
indexerror len of =  4
612 https://miuz.ru/catalog/rings/R4150-S-LRCJ3315SR17/
indexerror len of =  4
613 https://miuz.ru/catalog/rings/R4150-S-LR19324S/
614 https://miuz.ru/catalog/pendants/P4150-D-BP20

...

3437 https://miuz.ru/catalog/earrings/E4150-D-ERB2408S/
3438 https://miuz.ru/catalog/rings/R4150-D-46661E/
3439 https://miuz.ru/catalog/rings/R4150-D-46659E/
3440 https://miuz.ru/catalog/rings/R4150-D-R4433EM/
3441 https://miuz.ru/catalog/rings/R4150-C-LR285S/
3442 https://miuz.ru/catalog/earrings/E4150-C-1332371S/
3443 https://miuz.ru/catalog/rings/R4150-C-1333412R/
3444 https://miuz.ru/catalog/rings/R755-69004R001-R17/
3445 https://miuz.ru/catalog/rings/R4150-D-51053EM/
3446 https://miuz.ru/catalog/rings/R4150-D-47914E/
3447 https://miuz.ru/catalog/rings/R4150-D-LR12197E/
3448 https://miuz.ru/catalog/rings/R4150-C-1333412S/
3449 https://miuz.ru/catalog/earrings/E4150-C-ERP30310S/
3450 https://miuz.ru/catalog/rings/R4150-C-LR12194S/
3451 https://miuz.ru/catalog/rings/R4150-C-LR896E/
3452 https://miuz.ru/catalog/rings/R131-R4459-SA/
3453 https://miuz.ru/catalog/rings/R4150-D-48458EM/
3454 https://miuz.ru/catalog/rings/R4150-D-47918E/
3455 https://miuz.ru/catalog/rings/R4150-C-1333411E/

3589 https://miuz.ru/catalog/rings/R4150-D-46659E/
3590 https://miuz.ru/catalog/rings/R4150-D-R4433EM/
3591 https://miuz.ru/catalog/rings/R4150-C-LR285S/
3592 https://miuz.ru/catalog/earrings/E4150-C-1332371S/
3593 https://miuz.ru/catalog/rings/R4150-C-1333412R/
3594 https://miuz.ru/catalog/rings/R755-69004R001-R17/
3595 https://miuz.ru/catalog/rings/R4150-D-51053EM/
3596 https://miuz.ru/catalog/rings/R4150-D-47914E/
3597 https://miuz.ru/catalog/rings/R4150-D-LR12197E/
3598 https://miuz.ru/catalog/rings/R4150-C-1333412S/
3599 https://miuz.ru/catalog/earrings/E4150-C-ERP30310S/
3600 https://miuz.ru/catalog/rings/R4150-C-LR12194S/
3601 https://miuz.ru/catalog/rings/R4150-C-LR896E/
3602 https://miuz.ru/catalog/rings/R131-R4459-SA/
3603 https://miuz.ru/catalog/rings/R4150-D-48458EM/
3604 https://miuz.ru/catalog/rings/R4150-D-47918E/
3605 https://miuz.ru/catalog/rings/R4150-C-1333411E/
3606 https://miuz.ru/catalog/rings/R4150-C-LRP30310S/
3607 https://miuz.ru/catalog/rings/R4132-02-LR4278E/

3741 https://miuz.ru/catalog/rings/R4150-C-LR285S/
3742 https://miuz.ru/catalog/earrings/E4150-C-1332371S/
3743 https://miuz.ru/catalog/rings/R4150-C-1333412R/
3744 https://miuz.ru/catalog/rings/R755-69004R001-R17/
3745 https://miuz.ru/catalog/rings/R4150-D-51053EM/
3746 https://miuz.ru/catalog/rings/R4150-D-47914E/
3747 https://miuz.ru/catalog/rings/R4150-D-LR12197E/
3748 https://miuz.ru/catalog/rings/R4150-C-1333412S/
3749 https://miuz.ru/catalog/earrings/E4150-C-ERP30310S/
3750 https://miuz.ru/catalog/rings/R4150-C-LR12194S/
3751 https://miuz.ru/catalog/rings/R4150-C-LR896E/
3752 https://miuz.ru/catalog/rings/R131-R4459-SA/
3753 https://miuz.ru/catalog/rings/R4150-D-48458EM/
3754 https://miuz.ru/catalog/rings/R4150-D-47918E/
3755 https://miuz.ru/catalog/rings/R4150-C-1333411E/
3756 https://miuz.ru/catalog/rings/R4150-C-LRP30310S/
3757 https://miuz.ru/catalog/rings/R4132-02-LR4278E/
3758 https://miuz.ru/catalog/rings/R131-R4346-EM/
3759 https://miuz.ru/catalog/earrings/E2018-ERU1078A

3893 https://miuz.ru/catalog/rings/R4150-C-1333412R/
3894 https://miuz.ru/catalog/rings/R755-69004R001-R17/
3895 https://miuz.ru/catalog/rings/R4150-D-51053EM/
3896 https://miuz.ru/catalog/rings/R4150-D-47914E/
3897 https://miuz.ru/catalog/rings/R4150-D-LR12197E/
3898 https://miuz.ru/catalog/rings/R4150-C-1333412S/
3899 https://miuz.ru/catalog/earrings/E4150-C-ERP30310S/
3900 https://miuz.ru/catalog/rings/R4150-C-LR12194S/
3901 https://miuz.ru/catalog/rings/R4150-C-LR896E/
3902 https://miuz.ru/catalog/rings/R131-R4459-SA/
3903 https://miuz.ru/catalog/rings/R4150-D-48458EM/
3904 https://miuz.ru/catalog/rings/R4150-D-47918E/
3905 https://miuz.ru/catalog/rings/R4150-C-1333411E/
3906 https://miuz.ru/catalog/rings/R4150-C-LRP30310S/
3907 https://miuz.ru/catalog/rings/R4132-02-LR4278E/
3908 https://miuz.ru/catalog/rings/R131-R4346-EM/
3909 https://miuz.ru/catalog/earrings/E2018-ERU1078ASA-R17/
3910 https://miuz.ru/catalog/earrings/E108-1982669CXDS-R17/
3911 https://miuz.ru/catalog/rings/R108

4045 https://miuz.ru/catalog/rings/R4150-D-51053EM/
4046 https://miuz.ru/catalog/rings/R4150-D-47914E/
4047 https://miuz.ru/catalog/rings/R4150-D-LR12197E/
4048 https://miuz.ru/catalog/rings/R4150-C-1333412S/
4049 https://miuz.ru/catalog/earrings/E4150-C-ERP30310S/
4050 https://miuz.ru/catalog/rings/R4150-C-LR12194S/
4051 https://miuz.ru/catalog/rings/R4150-C-LR896E/
4052 https://miuz.ru/catalog/rings/R131-R4459-SA/
4053 https://miuz.ru/catalog/rings/R4150-D-48458EM/
4054 https://miuz.ru/catalog/rings/R4150-D-47918E/
4055 https://miuz.ru/catalog/rings/R4150-C-1333411E/
4056 https://miuz.ru/catalog/rings/R4150-C-LRP30310S/
4057 https://miuz.ru/catalog/rings/R4132-02-LR4278E/
4058 https://miuz.ru/catalog/rings/R131-R4346-EM/
4059 https://miuz.ru/catalog/earrings/E2018-ERU1078ASA-R17/
4060 https://miuz.ru/catalog/earrings/E108-1982669CXDS-R17/
4061 https://miuz.ru/catalog/rings/R108-1982526AS-R17/
4062 https://miuz.ru/catalog/bracelets/B764-BL118083-RU/
4063 https://miuz.ru/catalog/earrin

## save res

In [21]:
import os

In [22]:
folders = os.listdir()
[q for q in (enumerate(folders))]

[(0, '.DS_Store'),
 (1, '.git'),
 (2, '.idea'),
 (3, '.ipynb_checkpoints'),
 (4, 'AnaliticRep031221.ipynb'),
 (5, 'AnaliticRep060622EPL_Кольца_с_брилл-62.ipynb'),
 (6, 'AnaliticRep061221-KDM1C8.ipynb'),
 (7, 'AnaliticRep140422MUIZ_Кольца_с_брилл.ipynb'),
 (8, 'AnaliticRep220422EGELGE_брилл.ipynb'),
 (9, 'AnaliticRep240522MUIZ_Кольца_с_брилл.ipynb'),
 (10, 'data.xlsx'),
 (11, 'Egelge_1after_parsing.ipynb'),
 (12, 'env'),
 (13, 'for_parsing_func.ipynb'),
 (14, 'jewels'),
 (15, 'MUIZ_justcode.ipynb'),
 (16, 'Parsing.html'),
 (17, 'ParsingEgelge.ipynb'),
 (18, 'ParsingEPL.ipynb'),
 (19, 'ParsingKierge.ipynb'),
 (20, 'ParsingMiuz.ipynb'),
 (21, 'ParsingNKObyAita.ipynb'),
 (22, 'ParsingSUNLIGHT.ipynb'),
 (23, 'pars_16-06-2022_muiz_RSE.xlsx'),
 (24, 'README.md'),
 (25, 'requiremnts.txt'),
 (26, 'results'),
 (27, 'resultslinks4154_2410522.xlsx'),
 (28, 'rusprice'),
 (29, 'temp10.xlsx'),
 (30, 'Untitled.ipynb'),
 (31, 'Untitled1-Copy1.ipynb'),
 (32, 'Untitled1.ipynb'),
 (33, 'visio_matplotlib.i

In [23]:
resfolder = folders[26]


In [107]:
resfolder

'results'

In [115]:
data.to_excel(resfolder+'/'+'sunlight_bk_30082022.xlsx')


In [ ]:
#06-07-2022 серебро
#16-06-2022 Для Алексея Сапфир Изумруд Рубин
#24-05-2022
#14-04-2022

for i in range(0,5):# len(links)):
    try:
        urlcard = domain +links[i]
        card = mysoup(urlcard)
        row = len(data)
        data.loc[row,'url'] = urlcard
        data.loc[row,'h1'] = card.find("h1").text.strip()
        data.loc[row,'art'] = card.select(
            'body > div.grid > div > div > div.shops__breadcrumbs.catalog_breadcrumbs > div > div:nth-child(4) > span > span')[0].text
        data.loc[row,'price'] = float(''.join(re.findall(r'\d+', card.find(
            "div","places-cont__price b-price").find("div", 'new').text)))
        
        #data.loc[row,'price'] = float(''.join(re.findall(r'\d+', card.find('div',"detail__item-price-new").text)))
        data.loc[row,'gold'] = card.find_all('span','href')[2].text.strip()
        #data.loc[row,'gold'] = re.findall(r"(Золото\s+\d+)",str(card))[0]
        data.loc[row,'weight'] = float(re.findall(r"(?:Вес:</span>\n)\s+(\d+.\d+)",str(card))[0])
        #div_gems = card.select('body > div.grid > div > div > div.detail__item.detail__item--new > div.detail__item-col._3.position-relative > form > div:nth-child(8)')
        #data.loc[row,'gems'] = re.search(r'Камни:\s+([\w\W]+)', str(div_gems[0].text.strip()))[1]
        div_gems = card.select('body > div.grid > div > div > div.detail__item.detail__item--new > div.detail__item-col._3.position-relative > form > div:nth-child(8)')
        data.loc[row,'gems'] = re.findall(r'Камни:\s+([\w\W]+)', div_gems[0].text)
        
    except (AttributeError,TypeError) as e:
        print('error',e)
    except IndexError:
        print('indexerror len of = ' , len(card.find_all('div','detail__item-option')) )
    print(i, urlcard)

## Загрузка обработка

In [269]:
df = pd.read_excel('results/mius_серебро790_06072022.xlsx').drop(columns='Unnamed: 0')

In [270]:
df.columns

Index(['h1', 'art', 'price', 'gold', 'weight', 'gems', 'url'], dtype='object')

In [271]:
df.dropna(subset=['weight'], inplace = True)

In [107]:
#t = 'Серьги c фианитами E2029-1210015246-Z'
#t = 'Кольцо c фианитами R4150-S-LR19653'
t2 = 'Кольцо c жемчугом, сапфирами и фианитами R4150-S-LRP34769S'
t = 'Кольцо fdfd'

In [272]:
patt_h1_0 = r'([\w\W]+)(?= [A-Z]\d)+?'

In [ ]:
 r'([\w\W]+)(?= [A-Z]\d)+?'

In [273]:
re.search(patt_h1_0, t2)[1].strip()

'Кольцо c жемчугом, сапфирами и фианитами'

In [274]:
re.search( r'([\w\W]+)', t)[1].strip()

'Кольцо fdfd'

In [279]:
df['h1_0'] = df.h1.apply(lambda x: re.search(patt, x)[1] if re.search(patt, x) else x)

In [277]:
patt = patt_h1_0
for n,x in enumerate(df.h1):
    print(n, x,' -- ', (lambda x: re.search(patt, x)[1] if re.search(patt, x) else x)(x))#.strip())

0 Серьги c фианитами E2029-1210015246-Z  --  Серьги c фианитами
1 Серьги E2031-94140069  --  Серьги
2 Серьги E2029-1200016922  --  Серьги
3 Кольцо c эмалью R4150-S-R906A13124LG  --  Кольцо c эмалью
4 Серьги c фианитами E2036-SL-3752  --  Серьги c фианитами
5 Браслет c фианитами B758-12030029  --  Браслет c фианитами
6 Кольцо c кристаллом и фианитами R2029-1015014454-SC  --  Кольцо c кристаллом и фианитами
7 Серьги c эмалью E2019-94021423-EL  --  Серьги c эмалью
8 Подвеска c изумрудом P01-SILVER-015-EM  --  Подвеска c изумрудом
9 Брошь c сваровски H149-34600  --  Брошь c сваровски
10 Серьги c фианитами E2029-1210015290-Z  --  Серьги c фианитами
11 Кольцо c жемчугом R4150-S-LRKTR0033PR  --  Кольцо c жемчугом
12 Серьги E2031-94140045  --  Серьги
13 Браслет B2030-KL040CH-12022  --  Браслет
14 Колье N30-NH22-1628  --  Колье
15 Брошь c фианитами H2029-1510015796-ZI  --  Брошь c фианитами
16 Кольцо c жемчугом и фианитами R4150-S-LRKTR0129PR  --  Кольцо c жемчугом и фианитами
17 Серьги c фиани

712 Кольцо c фианитами R4150-S-LR00114  --  Кольцо c фианитами
713 Кольцо c фианитом R4150-S-LRP30324  --  Кольцо c фианитом
714 Цепь C4207-SN-007-017-Y  --  Цепь
715 Серьги c жемчугом и фианитами E758-1101100900  --  Серьги c жемчугом и фианитами
716 Серьги c стеклом E377-MPSETMULTI03  --  Серьги c стеклом
717 Серьги c фианитами E2036-SL-3717  --  Серьги c фианитами
718 Серьги c фианитами E183-CUR.ER.RED  --  Серьги c фианитами
719 Серьги c бриллиантами E01-SDI-35366  --  Серьги c бриллиантами
720 Серьги c жемчугом, сапфирами и фианитами E4150-S-ERP34766SR17  --  Серьги c жемчугом, сапфирами и фианитами
721 Браслет B2030-CLK020CH-21  --  Браслет
722 Серьги c фианитами E2029-1210015269-Z  --  Серьги c фианитами
723 Кольцо c изумрудами и фианитами R4150-S-LRP35163E  --  Кольцо c изумрудами и фианитами
724 Кольцо c фианитами R4150-S-LR00124  --  Кольцо c фианитами
725 Серьги E2031-94-123-01348-1  --  Серьги
726 Подвеска c фианитами P4150-S-BP19091  --  Подвеска c фианитами
727 Брошь c им

In [38]:
str(patt_h1_0)

'([\\w\\W]+)[A-z]\\d{2,}'

In [280]:
df.loc[:]

,h1,art,price,gold,weight,gems,url,h1_0
0,Серьги c фианитами E2029-1210015246-Z,Артикул E2029-1210015246-Z,855.0,Серебро 925,1.35,"['4 Фианита, 0.08 карат; ']",https://miuz.ru/catalog/earrings/E2029-1210015246-Z/,Серьги c фианитами
1,Серьги E2031-94140069,Артикул E2031-94140069,531.0,Серебро 925,0.53,[],https://miuz.ru/catalog/earrings/E2031-94140069/,Серьги
2,Серьги E2029-1200016922,Артикул E2029-1200016922,3195.0,Серебро 925,4.81,[],https://miuz.ru/catalog/earrings/E2029-1200016922/,Серьги
3,Кольцо c эмалью R4150-S-R906A13124LG,Артикул R4150-S-R906A13124LG,2048.0,Как измерить размер?,3.08,"['1 Эмаль, 0.01 карат; ']",https://miuz.ru/catalog/rings/R4150-S-R906A13124LG/,Кольцо c эмалью
4,Серьги c фианитами E2036-SL-3752,Артикул E2036-SL-3752,2061.0,Серебро 925,1.70,"['38 Фианитов огранки ""Круглая"", 0.34 карат; ']",https://miuz.ru/catalog/earrings/E2036-SL-3752/,Серьги c фианитами
...,...,...,...,...,...,...,...,...
785,Серьги c фианитами E01-SILVER-025-4.0-Z,Артикул E01-SILVER-025-4.0-Z,1226.0,Серебро 925,1.06,"['2 Фианита огранки ""Круглая"", 0.95 карат; ']",https://miuz.ru/catalog/earrings/E01-SILVER-025-4.0-Z/,Серьги c фианитами
786,Цепь C191-81055141450,Артикул C191-81055141450,1161.0,Серебро 925,4.37,[],https://miuz.ru/catalog/chain/C191-81055141450/,Цепь
787,Кольцо c фианитами R4150-S-LR19798,Артикул R4150-S-LR19798,8991.0,Как измерить размер?,13.52,"['373 Фианита, 6.281 карат; ']",https://miuz.ru/catalog/rings/R4150-S-LR19798/,Кольцо c фианитами
788,Кольцо c фианитами R2029-1010015270-Z,Артикул R2029-1010015270-Z,1161.0,Как измерить размер?,1.99,"['17 Фианитов, 0.61 карат; ']",https://miuz.ru/catalog/rings/R2029-1010015270-Z/,Кольцо c фианитами


In [281]:
df['gemsstr'] = df['gems'].str.replace('\[|\]','').str.strip("'")

df['gemslist'] = df.gemsstr.str.split(';')

<ipython-input-281-a67998840f91>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['gemsstr'] = df['gems'].str.replace('\[|\]','').str.strip("'")


In [282]:
df['gemslist'].map(len).unique()

array([2, 1, 3, 4, 5, 7, 8], dtype=int64)

In [283]:
df['lengem'] = df['gemslist'].map(len)

In [284]:
df.loc[df['gemslist'].map(len) <= 2, ['gems','gemslist','lengem']]

,gems,gemslist,lengem
0,"['4 Фианита, 0.08 карат; ']","[4 Фианита, 0.08 карат, ]",2
1,[],[],1
2,[],[],1
3,"['1 Эмаль, 0.01 карат; ']","[1 Эмаль, 0.01 карат, ]",2
4,"['38 Фианитов огранки ""Круглая"", 0.34 карат; ']","[38 Фианитов огранки ""Круглая"", 0.34 карат, ]",2
...,...,...,...
783,"['31 Фианит, 0.352 карат; ']","[31 Фианит, 0.352 карат, ]",2
785,"['2 Фианита огранки ""Круглая"", 0.95 карат; ']","[2 Фианита огранки ""Круглая"", 0.95 карат, ]",2
786,[],[],1
787,"['373 Фианита, 6.281 карат; ']","[373 Фианита, 6.281 карат, ]",2


In [286]:
sum(df['gemslist'].map(len) > 2)

150

In [290]:
df['gem01'] = df['gemslist'].apply(lambda x: x[0])

In [287]:
df.columns

Index(['h1', 'art', 'price', 'gold', 'weight', 'gems', 'url', 'h1_0',
       'gemsstr', 'gemslist', 'lengem'],
      dtype='object')

In [293]:
df01 = df.loc[df['gemslist'].map(len) <= 2, ['h1_0','h1','gems', 'price', 'gold', 'weight','gem01', 'url','lengem']]

In [294]:
df01

,h1_0,h1,gems,price,gold,weight,gem01,url,lengem
0,Серьги c фианитами,Серьги c фианитами E2029-1210015246-Z,"['4 Фианита, 0.08 карат; ']",855.0,Серебро 925,1.35,"4 Фианита, 0.08 карат",https://miuz.ru/catalog/earrings/E2029-1210015246-Z/,2
1,Серьги,Серьги E2031-94140069,[],531.0,Серебро 925,0.53,,https://miuz.ru/catalog/earrings/E2031-94140069/,1
2,Серьги,Серьги E2029-1200016922,[],3195.0,Серебро 925,4.81,,https://miuz.ru/catalog/earrings/E2029-1200016922/,1
3,Кольцо c эмалью,Кольцо c эмалью R4150-S-R906A13124LG,"['1 Эмаль, 0.01 карат; ']",2048.0,Как измерить размер?,3.08,"1 Эмаль, 0.01 карат",https://miuz.ru/catalog/rings/R4150-S-R906A13124LG/,2
4,Серьги c фианитами,Серьги c фианитами E2036-SL-3752,"['38 Фианитов огранки ""Круглая"", 0.34 карат; ']",2061.0,Серебро 925,1.70,"38 Фианитов огранки ""Круглая"", 0.34 карат",https://miuz.ru/catalog/earrings/E2036-SL-3752/,2
...,...,...,...,...,...,...,...,...,...
783,Кольцо c фианитами,Кольцо c фианитами R4150-S-LRL102123,"['31 Фианит, 0.352 карат; ']",878.0,Как измерить размер?,1.32,"31 Фианит, 0.352 карат",https://miuz.ru/catalog/rings/R4150-S-LRL102123/,2
785,Серьги c фианитами,Серьги c фианитами E01-SILVER-025-4.0-Z,"['2 Фианита огранки ""Круглая"", 0.95 карат; ']",1226.0,Серебро 925,1.06,"2 Фианита огранки ""Круглая"", 0.95 карат",https://miuz.ru/catalog/earrings/E01-SILVER-025-4.0-Z/,2
786,Цепь,Цепь C191-81055141450,[],1161.0,Серебро 925,4.37,,https://miuz.ru/catalog/chain/C191-81055141450/,1
787,Кольцо c фианитами,Кольцо c фианитами R4150-S-LR19798,"['373 Фианита, 6.281 карат; ']",8991.0,Как измерить размер?,13.52,"373 Фианита, 6.281 карат",https://miuz.ru/catalog/rings/R4150-S-LR19798/,2


In [329]:
t = '38 Фианитов огранки "Круглая", 0.34 карат'

In [332]:
dic={}

In [334]:
re.match(r'\d+',t)[0] 
dic['шт_вст'] = r'\d+'

In [339]:

dic['карат_вст'] = r'[\d.\d]+(?= карат)'
re.search(dic['карат_вст'],t)[0]

'0.34'

In [340]:
dic['описание_вст'] = r'(?!\d+).+(?=,)'
re.search(dic['описание_вст'],t)[0]

' Фианитов огранки "Круглая"'

In [346]:
for c in dic.items():
    print(c)
    df01[c[0]] = df01.gem01.apply(lambda x: re.search(c[1],x)[0] if re.search(c[1],x) else '')

('шт_вст', '\\d+')
('карат_вст', '[\\d.\\d]+(?= карат)')
('описание_вст', '(?!\\d+).+(?=,)')


In [328]:
for n, x in enumerate(df01.gem01):
    print(n,x,'---> ', re.search(r'[\d.\d]+(?= карат)',x)[0] if re.search(r'[\d.\d]+(?= карат)',x) else '')

0 4  Фианита, 0.08 карат --->  0.08
1  --->  
2  --->  
3 1  Эмаль, 0.01 карат --->  0.01
4 38  Фианитов огранки "Круглая", 0.34 карат --->  0.34
5 78  Фианитов огранки "Круглая", 1.85 карат --->  1.85
6 2  Эмали, 0.1 карат --->  0.1
7 1  Изумруд огранки "Фантазия", цвет 3, чистота 3, 0.01 карат --->  0.01
8 62  Сваровски, 0.1 карат --->  0.1
9 2  Фианита огранки "Круглая", 0.84 карат --->  0.84
10 1  Жемчуг, 3.27 карат --->  3.27
11  --->  
12  --->  
13  --->  
14 15  Фианитов, 0.19 карат --->  0.19
15 163  Фианита огранки "Круглая", 7.6 карат --->  7.6
16 14  Фианитов, 0.64 карат --->  0.64
17 1  Эмаль, 0.01 карат --->  0.01
18  --->  
19 8  Фианитов, 1.73 карат --->  1.73
20 60  Сваровски, 0.1 карат --->  0.1
21 2  Фианита, 0.84 карат --->  0.84
22  --->  
23  --->  
24 44  Вставки из жемчуга, Круглая огранка, 378.9 карат --->  378.9
25 2  Бриллианта, Круглая огранка, тип огранки A, цвет 3, чистота 4, 0.02 карат --->  0.02
26 7  Фианитов, 0.76 карат --->  0.76
27 10  Сваровски огра

In [347]:
df01

,h1_0,h1,gems,price,gold,weight,gem01,url,lengem,шт_вст,карат_вст,описание_вст
0,Серьги c фианитами,Серьги c фианитами E2029-1210015246-Z,"['4 Фианита, 0.08 карат; ']",855.0,Серебро 925,1.35,"4 Фианита, 0.08 карат",https://miuz.ru/catalog/earrings/E2029-1210015246-Z/,2,4,0.08,Фианита
1,Серьги,Серьги E2031-94140069,[],531.0,Серебро 925,0.53,,https://miuz.ru/catalog/earrings/E2031-94140069/,1,,,
2,Серьги,Серьги E2029-1200016922,[],3195.0,Серебро 925,4.81,,https://miuz.ru/catalog/earrings/E2029-1200016922/,1,,,
3,Кольцо c эмалью,Кольцо c эмалью R4150-S-R906A13124LG,"['1 Эмаль, 0.01 карат; ']",2048.0,Как измерить размер?,3.08,"1 Эмаль, 0.01 карат",https://miuz.ru/catalog/rings/R4150-S-R906A13124LG/,2,1,0.01,Эмаль
4,Серьги c фианитами,Серьги c фианитами E2036-SL-3752,"['38 Фианитов огранки ""Круглая"", 0.34 карат; ']",2061.0,Серебро 925,1.70,"38 Фианитов огранки ""Круглая"", 0.34 карат",https://miuz.ru/catalog/earrings/E2036-SL-3752/,2,38,0.34,"Фианитов огранки ""Круглая"""
...,...,...,...,...,...,...,...,...,...,...,...,...
783,Кольцо c фианитами,Кольцо c фианитами R4150-S-LRL102123,"['31 Фианит, 0.352 карат; ']",878.0,Как измерить размер?,1.32,"31 Фианит, 0.352 карат",https://miuz.ru/catalog/rings/R4150-S-LRL102123/,2,31,0.352,Фианит
785,Серьги c фианитами,Серьги c фианитами E01-SILVER-025-4.0-Z,"['2 Фианита огранки ""Круглая"", 0.95 карат; ']",1226.0,Серебро 925,1.06,"2 Фианита огранки ""Круглая"", 0.95 карат",https://miuz.ru/catalog/earrings/E01-SILVER-025-4.0-Z/,2,2,0.95,"Фианита огранки ""Круглая"""
786,Цепь,Цепь C191-81055141450,[],1161.0,Серебро 925,4.37,,https://miuz.ru/catalog/chain/C191-81055141450/,1,,,
787,Кольцо c фианитами,Кольцо c фианитами R4150-S-LR19798,"['373 Фианита, 6.281 карат; ']",8991.0,Как измерить размер?,13.52,"373 Фианита, 6.281 карат",https://miuz.ru/catalog/rings/R4150-S-LR19798/,2,373,6.281,Фианита


In [348]:
resfolder

'results'

In [349]:
df01.to_excel(resfolder+'/'+'нефть1сорт_серебро_мюз_604_06-07-2022.xlsx')

In [255]:
df['IDizd'] = df.index

In [259]:
df.columns

Index(['h1', 'art', 'price', 'gold', 'weight', 'gems', 'url', 'h1_0',
       'gemsstr', 'gemslist', 'IDizd'],
      dtype='object')

In [262]:
df.explode("gemslist")  #[["IDizd","gemslist"]]

,h1,art,price,gold,weight,gems,url,h1_0,gemsstr,gemslist,IDizd
0,Серьги c фианитами E2029-1210015246-Z,Артикул E2029-1210015246-Z,855.0,Серебро 925,1.35,"['4 Фианита, 0.08 карат; ']",https://miuz.ru/catalog/earrings/E2029-1210015246-Z/,Серьги c фианитами,"4 Фианита, 0.08 карат;","4 Фианита, 0.08 карат",0
0,Серьги c фианитами E2029-1210015246-Z,Артикул E2029-1210015246-Z,855.0,Серебро 925,1.35,"['4 Фианита, 0.08 карат; ']",https://miuz.ru/catalog/earrings/E2029-1210015246-Z/,Серьги c фианитами,"4 Фианита, 0.08 карат;",,0
1,Серьги E2031-94140069,Артикул E2031-94140069,531.0,Серебро 925,0.53,[],https://miuz.ru/catalog/earrings/E2031-94140069/,Серьги,,,1
2,Серьги E2029-1200016922,Артикул E2029-1200016922,3195.0,Серебро 925,4.81,[],https://miuz.ru/catalog/earrings/E2029-1200016922/,Серьги,,,2
3,Кольцо c эмалью R4150-S-R906A13124LG,Артикул R4150-S-R906A13124LG,2048.0,Как измерить размер?,3.08,"['1 Эмаль, 0.01 карат; ']",https://miuz.ru/catalog/rings/R4150-S-R906A13124LG/,Кольцо c эмалью,"1 Эмаль, 0.01 карат;","1 Эмаль, 0.01 карат",3
...,...,...,...,...,...,...,...,...,...,...,...
788,Кольцо c фианитами R2029-1010015270-Z,Артикул R2029-1010015270-Z,1161.0,Как измерить размер?,1.99,"['17 Фианитов, 0.61 карат; ']",https://miuz.ru/catalog/rings/R2029-1010015270-Z/,Кольцо c фианитами,"17 Фианитов, 0.61 карат;","17 Фианитов, 0.61 карат",788
788,Кольцо c фианитами R2029-1010015270-Z,Артикул R2029-1010015270-Z,1161.0,Как измерить размер?,1.99,"['17 Фианитов, 0.61 карат; ']",https://miuz.ru/catalog/rings/R2029-1010015270-Z/,Кольцо c фианитами,"17 Фианитов, 0.61 карат;",,788
789,Колье c жемчугом и фианитами N4150-S-BRKTP0106PR,Артикул N4150-S-BRKTP0106PR,11391.0,Серебро 925,3.67,"['1 Жемчуг, 6.03 карат;21 Фианит, 0.62 карат; ']",https://miuz.ru/catalog/necklace/N4150-S-BRKTP0106PR/,Колье c жемчугом и фианитами,"1 Жемчуг, 6.03 карат;21 Фианит, 0.62 карат;","1 Жемчуг, 6.03 карат",789
789,Колье c жемчугом и фианитами N4150-S-BRKTP0106PR,Артикул N4150-S-BRKTP0106PR,11391.0,Серебро 925,3.67,"['1 Жемчуг, 6.03 карат;21 Фианит, 0.62 карат; ']",https://miuz.ru/catalog/necklace/N4150-S-BRKTP0106PR/,Колье c жемчугом и фианитами,"1 Жемчуг, 6.03 карат;21 Фианит, 0.62 карат;","21 Фианит, 0.62 карат",789


### Отступление, о том как это превратилось в текст и когда, и как я обратно в лист

In [2]:
from bs4 import BeautifulSoup
import requests
from requests import Request, Session
import pandas as pd

In [3]:
import re

In [174]:
def mysoup(urls):
    response = requests.get(urls)
    return BeautifulSoup(response.text, 'lxml')   

In [218]:
card = mysoup('https://miuz.ru/catalog/rings/R4150-S-LRP34775CP/')

In [219]:
div_gems = card.select('body > div.grid > div > \
div > div.detail__item.detail__item--new > \
div.detail__item-col._3.position-relative >\
form > div:nth-child(8)')
a = re.findall(r'Камни:\s+([\w\W]+)', div_gems[0].text)
a, type(a)

(['15  Фианитов, 0.093 карат;1  Жемчуг, 6.265 карат; '], list)

In [220]:
data = pd.DataFrame(columns=['h1','art','price','gold','weight','gems','url'])
data.loc[len(data),'gems'] = a

In [233]:
data['gems']

0    [15  Фианитов, 0.093 карат;1  Жемчуг, 6.265 карат; ]
Name: gems, dtype: object

In [222]:
data.loc[0,'gems'], type(data.loc[0,'gems'])

(['15  Фианитов, 0.093 карат;1  Жемчуг, 6.265 карат; '], list)

In [223]:
data.to_excel('data.xlsx', index=None)

In [224]:
data_e = pd.read_excel('data.xlsx')

In [232]:
data_e['gems']

0    ['15  Фианитов, 0.093 карат;1  Жемчуг, 6.265 карат; ']
Name: gems, dtype: object

In [226]:
data_e.loc[0,'gems'] , type(df.loc[0,'gems'])

("['15  Фианитов, 0.093 карат;1  Жемчуг, 6.265 карат; ']", str)

In [227]:
data_e['gems_list'] = data_e['gems'].str.replace('\[|\]','').str.strip("'").str.split(';')

<ipython-input-227-a081dc441475>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data_e['gems_list'] = data_e['gems'].str.replace('\[|\]','').str.strip("'").str.split(';')


In [230]:
pd.options.display.max_colwidth = 200

In [231]:
data_e[['gems','gems_list']]

,gems,gems_list
0,"['15 Фианитов, 0.093 карат;1 Жемчуг, 6.265 карат; ']","[15 Фианитов, 0.093 карат, 1 Жемчуг, 6.265 карат, ]"


In [234]:
data_e.loc[0,'gems_list'] , type(data_e.loc[0,'gems_list'])

(['15  Фианитов, 0.093 карат', '1  Жемчуг, 6.265 карат', ' '], list)

In [ ]:
### Отступление, о том как это превратилось в текст и когда, и как я обратно в лист

from bs4 import BeautifulSoup
import requests
from requests import Request, Session
import pandas as pd

import re

def mysoup(urls):
    response = requests.get(urls)
    return BeautifulSoup(response.text, 'lxml')   

card = mysoup('https://miuz.ru/catalog/rings/R4150-S-LRP34775CP/')

div_gems = card.select('body > div.grid > div > \
div > div.detail__item.detail__item--new > \
div.detail__item-col._3.position-relative >\
form > div:nth-child(8)')
a = re.findall(r'Камни:\s+([\w\W]+)', div_gems[0].text)
a, type(a)

data = pd.DataFrame(columns=['h1','art','price','gold','weight','gems','url'])
data.loc[len(data),'gems'] = a

data['gems']

data.loc[0,'gems'], type(data.loc[0,'gems'])

data.to_excel('data.xlsx', index=None)

data_e = pd.read_excel('data.xlsx')

data_e['gems']

data_e.loc[0,'gems'] , type(data_e.loc[0,'gems'])

data_e['gems_list'] = data_e['gems'].str.replace('\[|\]','').str.strip("'").str.split(';')

pd.options.display.max_colwidth = 200

data_e[['gems','gems_list']]

data_e.loc[0,'gems_list'] , type(data_e.loc[0,'gems_list'])

In [237]:
df

,h1,art,price,gold,weight,gems,url,h1_0,gemsstr,gemslist
0,Серьги c фианитами E2029-1210015246-Z,Артикул E2029-1210015246-Z,855.0,Серебро 925,1.35,"['4 Фианита, 0.08 карат; ']",https://miuz.ru/catalog/earrings/E2029-1210015246-Z/,Серьги c фианитами,"4 Фианита, 0.08 карат;","[4 Фианита, 0.08 карат, ]"
1,Серьги E2031-94140069,Артикул E2031-94140069,531.0,Серебро 925,0.53,[],https://miuz.ru/catalog/earrings/E2031-94140069/,Серьги,,[]
2,Серьги E2029-1200016922,Артикул E2029-1200016922,3195.0,Серебро 925,4.81,[],https://miuz.ru/catalog/earrings/E2029-1200016922/,Серьги,,[]
3,Кольцо c эмалью R4150-S-R906A13124LG,Артикул R4150-S-R906A13124LG,2048.0,Как измерить размер?,3.08,"['1 Эмаль, 0.01 карат; ']",https://miuz.ru/catalog/rings/R4150-S-R906A13124LG/,Кольцо c эмалью,"1 Эмаль, 0.01 карат;","[1 Эмаль, 0.01 карат, ]"
4,Серьги c фианитами E2036-SL-3752,Артикул E2036-SL-3752,2061.0,Серебро 925,1.70,"['38 Фианитов огранки ""Круглая"", 0.34 карат; ']",https://miuz.ru/catalog/earrings/E2036-SL-3752/,Серьги c фианитами,"38 Фианитов огранки ""Круглая"", 0.34 карат;","[38 Фианитов огранки ""Круглая"", 0.34 карат, ]"
...,...,...,...,...,...,...,...,...,...,...
785,Серьги c фианитами E01-SILVER-025-4.0-Z,Артикул E01-SILVER-025-4.0-Z,1226.0,Серебро 925,1.06,"['2 Фианита огранки ""Круглая"", 0.95 карат; ']",https://miuz.ru/catalog/earrings/E01-SILVER-025-4.0-Z/,Серьги c фианитами,"2 Фианита огранки ""Круглая"", 0.95 карат;","[2 Фианита огранки ""Круглая"", 0.95 карат, ]"
786,Цепь C191-81055141450,Артикул C191-81055141450,1161.0,Серебро 925,4.37,[],https://miuz.ru/catalog/chain/C191-81055141450/,Цепь,,[]
787,Кольцо c фианитами R4150-S-LR19798,Артикул R4150-S-LR19798,8991.0,Как измерить размер?,13.52,"['373 Фианита, 6.281 карат; ']",https://miuz.ru/catalog/rings/R4150-S-LR19798/,Кольцо c фианитами,"373 Фианита, 6.281 карат;","[373 Фианита, 6.281 карат, ]"
788,Кольцо c фианитами R2029-1010015270-Z,Артикул R2029-1010015270-Z,1161.0,Как измерить размер?,1.99,"['17 Фианитов, 0.61 карат; ']",https://miuz.ru/catalog/rings/R2029-1010015270-Z/,Кольцо c фианитами,"17 Фианитов, 0.61 карат;","[17 Фианитов, 0.61 карат, ]"


In [135]:
df.drop(columns='gemsl', inplace=True)

In [154]:
s = "one [two] three [1] [2]"
print(re.sub('\[|\]', '', s))

one two three 1 2


# del

In [126]:
linkspd = pd.DataFrame(links)

In [127]:
linkspd.rename(columns={0:'short'}, inplace=True)

In [128]:
linkspd['path'] = domain +linkspd['short']

In [131]:
linkspd.to_excel(resfolder+'/'+'links4154_2410522.xlsx')

In [318]:
re.findall(r"(?:Вес:</span>\n)\s+(\d+.\d+)",str(card))

[]

In [ ]:
card = mysoup(urlcard)
row = len(data)
data.loc[row,'h1'] = card.find("h1").text.strip()
data.loc[row,'art'] = card.select('body > div.grid > div > div > div.shops__breadcrumbs.catalog_breadcrumbs > div > div:nth-child(4) > span > span')

data.loc[row,'price'] = float(''.join(re.findall(r'\d+', card.find('div',"detail__item-price-new").text)))
              

#quotes_inner = card.find_all('div', "detail__item-option")
#quotes_inner

data.loc[row,'gold'] = re.findall(r"(Золото\s+\d+)",str(quotes_inner[0]))[0]

data.loc[row,'weight'] = float(re.findall(r"(?:Вес:</span>\n)\s+(\d+.\d+)",str(card))[0])

div_gems = card.select('body > div.grid > div > div > div.detail__item.detail__item--new > div.detail__item-col._3.position-relative > form > div:nth-child(8)')

data.loc[row,'gems'] = re.search(r'Камни:\s+([\w\W]+)', str(div_gems[0].text.strip()))[1]
